Processed data from 23/03/2025 to 18/09/2025

In [88]:
import pandas as pd
path=pd.read_csv('/home/hieu/Work/new_casacom/result/path_compressed.csv')
home=pd.read_csv('/home/hieu/Work/new_casacom/result/home.csv')
work=pd.read_csv('/home/hieu/Work/new_casacom/result/work.csv')
leisure=pd.read_csv('/home/hieu/Work/new_casacom/result/leisure.csv')
maid_maping=pd.read_feather('/home/hieu/Work/maid_mapping.feather').to_dict(orient='records')

In [2]:
home.shape,work.shape,leisure.shape,path.shape

((686110, 78), (423420, 78), (674274, 78), (455315, 9))

In [89]:
# Calculate the number of unique MAIDs across all datasets
unique_maids = set().union(
    home.maid.unique(), 
    path.maid.unique(), 
    work.maid.unique(), 
    leisure.maid.unique()
)

# Print the count of unique MAIDs
print(f"Total Maids from 23-03-2025 to 18-09-2025: {len(maid_maping)}")
print(f"Number of unique MAIDs processed: {len(unique_maids)}")
print(f"percentage of unique MAIDs processed: {round(len(unique_maids)/len(maid_maping)*100,2)}%")

Total Maids from 23-03-2025 to 18-09-2025: 8954449
Number of unique MAIDs processed: 1305415
percentage of unique MAIDs processed: 14.58%


In [3]:
print(f"Total home maids: {home.shape[0]}")
print(f"Total work maids: {work.shape[0]}")
print(f"Total leisure maids: {leisure.shape[0]}")
print(f"Total path maids: {path.shape[0]}")
print(f"Total combined: {home.shape[0] + work.shape[0] + leisure.shape[0] + path.shape[0]}")

Total home maids: 686110
Total work maids: 423420
Total leisure maids: 674274
Total path maids: 455315
Total combined: 2239119


In [4]:
home_p,work_p,leisure_p=home[home.country=='Morocco'].shape[0]/home.shape[0]*100,work[work.country=='Morocco'].shape[0]/work.shape[0]*100,leisure[leisure.country=='Morocco'].shape[0]/leisure.shape[0]*100
print(f"Percentage of home points in Morocco: {home_p:.2f}%")
print(f"Percentage of work points in Morocco: {work_p:.2f}%")
print(f"Percentage of leisure points in Morocco: {leisure_p:.2f}%")


Percentage of home points in Morocco: 71.77%
Percentage of work points in Morocco: 69.13%
Percentage of leisure points in Morocco: 77.57%


In [5]:
# Calculate percentage of points in countries other than Morocco
other_home_p = 100 - home_p
other_work_p = 100 - work_p
other_leisure_p = 100 - leisure_p

print(f"Percentage of home points in other countries: {other_home_p:.2f}%")
print(f"Percentage of work points in other countries: {other_work_p:.2f}%")
print(f"Percentage of leisure points in other countries: {other_leisure_p:.2f}%")


Percentage of home points in other countries: 28.23%
Percentage of work points in other countries: 30.87%
Percentage of leisure points in other countries: 22.43%


In [ ]:
# Test Canonical MAID Mapping System

import pandas as pd
import os

# Load MAID tuple mapping từ file feather (giống như trong process_maroc.py)
maid_tuple_file = os.environ.get('MAID_TUPLE_FILE', './maid_tuple.feather')
print(f"Loading MAID tuple mapping from {maid_tuple_file}...")

# Read the maid_tuple file (first few rows for testing)
maid_tuple = pd.read_feather(maid_tuple_file)
print(f"Loaded {len(maid_tuple)} MAID pairs for testing")

# Show first 10 rows to understand the structure
print("\nFirst 10 MAID pairs:")
for i, (_, row) in enumerate(maid_tuple.head(10).iterrows()):
    canonical_maid = row[0]
    duplicate_maid = row[1]
    print(f"  {i+1}. Canonical: {canonical_maid[:50]}...")
    print(f"     Duplicate: {duplicate_maid[:50]}...")

# Create maid_mapping dictionary (giống như trong process_maroc.py)
maid_mapping = {}
valid_maids = set()

for _, row in maid_tuple.iterrows():
    canonical_maid = row[0]
    duplicate_maid = row[1]
    maid_mapping[canonical_maid] = canonical_maid
    maid_mapping[duplicate_maid] = canonical_maid
    valid_maids.add(canonical_maid)
    valid_maids.add(duplicate_maid)

print(f"\nCreated mapping for {len(maid_mapping)} MAIDs")
print(f"Total unique valid MAIDs: {len(valid_maids)}")

# Demo: Test mapping với một số MAID examples
print("\n" + "="*60)
print("DEMO: Testing Canonical MAID Mapping")
print("="*60)

# Lấy một vài ví dụ từ dữ liệu thực tế
test_maids = []
for _, row in maid_tuple.head(5).iterrows():
    test_maids.append(row[0])  # canonical
    test_maids.append(row[1])  # duplicate

print(f"Testing with {len(test_maids)} MAIDs:")
for i, test_maid in enumerate(test_maids):
    if test_maid in maid_mapping:
        canonical_result = maid_mapping[test_maid]
        print(f"  {i+1}. {test_maid[:50]}... → {canonical_result[:50]}...")
    else:
        print(f"  {i+1}. {test_maid[:50]}... → NOT FOUND")

# Demo với dữ liệu mẫu giả lập
print("\n" + "-"*40)
print("DEMO: Mapping in practice")
print("-"*40)

# Tạo sample data giả lập
sample_data = pd.DataFrame({
    'maid': [
        test_maids[0],  # canonical
        test_maids[1],  # duplicate
        'some_unknown_maid_12345',  # không có trong mapping
        test_maids[2],  # canonical
        test_maids[3]   # duplicate
    ],
    'latitude': [33.9716, 33.9716, 35.0, 34.0209, 34.0209],
    'longitude': [-6.8498, -6.8498, -5.0, -6.7933, -6.7933],
    'timestamp': ['2025-08-01', '2025-08-01', '2025-08-01', '2025-08-01', '2025-08-01']
})

print("Original sample data:")
print(sample_data[['maid']].head())

# Áp dụng canonical mapping
print("\nAfter canonical mapping:")
sample_data['canonical_maid'] = sample_data['maid'].map(maid_mapping)
print(sample_data[['maid', 'canonical_maid']].head())

# Đếm số lượng unique MAID trước và sau mapping
original_unique = sample_data['maid'].nunique()
canonical_unique = sample_data['canonical_maid'].dropna().nunique()

print(f"\nOriginal unique MAIDs: {original_unique}")
print(f"Canonical unique MAIDs: {canonical_unique}")
print(f"Reduction: {original_unique - canonical_unique} MAIDs consolidated")

print("\n" + "="*60)
print("Canonical MAID system helps consolidate duplicate device IDs")
print("into unique canonical identifiers for consistent analysis.")
print("="*60)


In [6]:
df=pd.concat([home,work,leisure])
df.reset_index(drop=True,inplace=True)
fluxes_pings=df.groupby('maid').agg({
    "maid_fluxB":"first",
    "maid_fluxC":"first",
    "maid_fluxD":"first",
    "maid_fluxE":"first",
    "maid_fluxF":"first",
})
b_pings=fluxes_pings.maid_fluxB.sum()
c_pings=fluxes_pings.maid_fluxC.sum()
d_pings=fluxes_pings.maid_fluxD.sum()
e_pings=fluxes_pings.maid_fluxE.sum()
f_pings=fluxes_pings.maid_fluxF.sum()
total_pings=b_pings+c_pings+d_pings+e_pings+f_pings
b_percent = b_pings/total_pings*100
c_percent = c_pings/total_pings*100
d_percent = d_pings/total_pings*100
e_percent = e_pings/total_pings*100
f_percent = f_pings/total_pings*100
# Display results
print(f"Flux B: {round(b_percent,2)}%")
print(f"Flux C: {round(c_percent,2)}%")
print(f"Flux D: {round(d_percent,2)}%")
print(f"Flux E: {round(e_percent,2)}%")
print(f"Flux F: {round(f_percent,2)}%")

Flux B: 64.24%
Flux C: 22.27%
Flux D: 11.51%
Flux E: 0.53%
Flux F: 1.44%


In [1]:
import pandas as pd
home_gt=pd.read_parquet('/home/hieu/Work/new_casacom/match_home_gt.parquet')
work_gt=pd.read_parquet('/home/hieu/Work/new_casacom/match_work_gt.parquet')
home_gt.shape,work_gt.shape

((45225, 4), (13916, 4))

In [2]:
home_maid=pd.read_parquet("/home/hieu/Work/new_casacom/match_home.parquet")
work_maid=pd.read_parquet("/home/hieu/Work/new_casacom/match_work.parquet")

In [10]:
# Print total counts
print(f"Total home MAIDs matched: {home_maid.maid.nunique()}")
print(f"Total work MAIDs matched: {work_maid.maid.nunique()}")

# Calculate match percentages
print("Our home Maids matched",home_gt.maid.nunique())
home_match_percent = home_gt.maid.nunique()/home_maid.maid.nunique()*100
print("Our work Maids matched",work_gt.maid.nunique())
work_match_percent = work_gt.maid.nunique()/work_maid.maid.nunique()*100

print(f"Home MAID match percentage: {home_match_percent:.2f}%")
print(f"Work MAID match percentage: {work_match_percent:.2f}%")
import pandas as pd
matched_home_89=pd.read_parquet('/home/hieu/Work/new_casacom/data/months/all_home_match.parquet')
matched_work_89=pd.read_parquet('/home/hieu/Work/new_casacom/data/months/all_work_match.parquet')
print(f"Number of MAIDs of F 04-05 matched with 8.9 million maids:\n -Home: {matched_home_89.shape[0]}\n -Work: {matched_work_89.shape[0]}")
# number of maid have home and work
# # Print the number of unique MAIDs that have home and work locations
# print(f"Our Number of unique MAIDs with home locations matched with 8.9 million maids: {home.maid.nunique()}")
# print(f"Our Number of unique MAIDs with work locations matched with 8.9 million maids: {work.maid.nunique()}")
# print(f"Our Number of unique MAIDs with home and work locations matched with 8.9 million maids: {len(set(home.maid.unique()) | set(work.maid.unique()))}")

Total home MAIDs matched: 25609
Total work MAIDs matched: 9890
Our home Maids matched 23585
Our work Maids matched 5914
Home MAID match percentage: 92.10%
Work MAID match percentage: 59.80%
Number of MAIDs of F 04-05 matched with 8.9 million maids:
 -Home: 37904
 -Work: 32175


In [3]:
import pandas as pd
all_maid=pd.read_parquet('/home/hieu/Work/new_casacom/data/months/all_maid.parquet')

In [18]:
count=all_maid.groupby('flux').agg({'maid_flux':'sum','maid':'nunique'}).reset_index()

In [66]:
count

,flux,maid_flux,maid
0,B,7.990753e+08,2387429
1,C,3.371785e+08,1001857
2,D,1.513642e+08,504140
3,E,5.370230e+09,8621992
4,F,3.565810e+07,400117


In [4]:
filltered_maid=all_maid[~all_maid.maid.isin(unique_maids)]

In [20]:
len(unique_maids)

1305415

In [72]:
all_maid[~all_maid.maid.isin(unique_maids)].groupby('flux').agg({'maid_flux':'sum','maid':'nunique'}).reset_index().to_dict(orient='records')

[{'flux': 'B', 'maid_flux': 44986993.0, 'maid': 1529428},
 {'flux': 'C', 'maid_flux': 20943398.0, 'maid': 415636},
 {'flux': 'D', 'maid_flux': 2116949.0, 'maid': 112570},
 {'flux': 'E', 'maid_flux': 3670690169.0, 'maid': 7552018},
 {'flux': 'F', 'maid_flux': 1801444.0, 'maid': 124836}]

In [73]:
a=[{'flux': 'B', 'total_pings': 799075289.0, 'maid': 2387429},
 {'flux': 'C', 'total_pings': 337178455.0, 'maid': 1001857},
 {'flux': 'D', 'total_pings': 151364159.0, 'maid': 504140},
 {'flux': 'E', 'total_pings': 5370230131.0, 'maid': 8621992},
 {'flux': 'F', 'total_pings': 35658096.0, 'maid': 400117}]

b=[{'flux': 'B', 'total_pings': 44986993.0, 'maid': 1529428},
 {'flux': 'C', 'total_pings': 20943398.0, 'maid': 415636},
 {'flux': 'D', 'total_pings': 2116949.0, 'maid': 112570},
 {'flux': 'E', 'total_pings': 3670690169.0, 'maid': 7552018},
 {'flux': 'F', 'total_pings': 1801444.0, 'maid': 124836}]

# Calculate percentage of b divided by a
percentages = []
for item_a in a:
    for item_b in b:
        if item_a['flux'] == item_b['flux']:
            percentages.append({
                'flux': item_a['flux'],
                'pings_percentage': (item_b['total_pings'] / item_a['total_pings']) * 100,
                'maid_percentage': (item_b['maid'] / item_a['maid']) * 100
            })
            break

# Display the percentages
for p in percentages:
    print(f"Flux {p['flux']}: Pings {p['pings_percentage']:.2f}%, MAIDs {p['maid_percentage']:.2f}%")

Flux B: Pings 5.63%, MAIDs 64.06%
Flux C: Pings 6.21%, MAIDs 41.49%
Flux D: Pings 1.40%, MAIDs 22.33%
Flux E: Pings 68.35%, MAIDs 87.59%
Flux F: Pings 5.05%, MAIDs 31.20%


In [86]:
dd=pd.concat([home,work,leisure,path], join='inner')
dd = dd[['confidence', 'category']]

In [89]:
# Create bins for confidence scores
bins = [0, 0.2, 0.5, 0.75, 0.9, 1.0]
labels = ['0-20%', '20-50%', '50-75%', '75-90%', '90-100%']

# Add a new column with binned confidence scores
dd['confidence_bin'] = pd.cut(dd['confidence'], bins=bins, labels=labels, right=True)

# Create a pivot table to count geohashes for each category and confidence bin
confidence_stats = pd.pivot_table(
    data=dd,
    index='category',
    columns='confidence_bin',
    aggfunc='nunique',
    fill_value=0
)

# Filter to show only home, work, leisure categories
categories_of_interest = ['home', 'work', 'leisure','path']
confidence_stats = confidence_stats.reindex(categories_of_interest)

# Calculate percentage distribution across confidence bins for each category
confidence_pct = confidence_stats.div(confidence_stats.sum(axis=1), axis=0) * 100

# Display the statistics tables
print("Count of unique geohashes by category and confidence:")
display(confidence_stats)

print("\nPercentage distribution by category and confidence:")
display(confidence_pct.round(2))

/tmp/ipykernel_5128/3826255278.py:9: FutureWarning:

The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior



Count of unique geohashes by category and confidence:


confidence                                
confidence_bin      0-20%  20-50%  50-75%  75-90% 90-100%
category                                                 
home                53566  209096  150229   29613    1841
work                44159  118569   33412    1645       0
leisure             74894  150364   31178      89       0
path                    0       0    8407  165576  235903


Percentage distribution by category and confidence:


confidence                             
confidence_bin      0-20% 20-50% 50-75% 75-90% 90-100%
category                                              
home                12.06  47.06  33.81   6.66    0.41
work                22.33  59.95  16.89   0.83    0.00
leisure             29.20  58.62  12.15   0.03    0.00
path                 0.00   0.00   2.05  40.40   57.55

In [18]:
hihi=matched_home_89[~matched_home_89.ad_id.isin(home_maid.ad_id_gt)]

In [33]:
import duckdb 
home_they=duckdb.query("""
select * from read_parquet('/home/hieu/Work/casacom/import/home/2025/**/*.parquet')
where ad_id in (select distinct ad_id from hihi)
""").df()



FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [ ]:
import duckdb
filtered_df=duckdb.query("""
select * from read_parquet('/home/hieu/Work/new_casacom/data/raw_rt/**/*.parquet')
where maid in (select distinct ad_id from hihi limit 10000)
""").df()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [24]:
drop=filtered_df.drop_duplicates(subset=['latitude','longitude'])

In [25]:
drop.shape[0]/filtered_df.shape[0]*100

30.19521702521014

In [26]:
drop.maid.nunique()/filtered_df.maid.nunique()*100  

73.67243133265514

In [46]:
drop

,maid,timestamp,date,country,latitude,longitude,horizontal_accuracy,ipv4,ipv6,altitude,altitude_accuracy,flux
0,iWojr9R/XwhOONU4pj/Nrg0OZJihc4UvSnDqVhNePtmMbb...,2025-08-01 08:03:17+07:00,2025-08-01,IDN,-7.641480,111.517807,24.0,139.228.90.177,,0.0,0.0,B
1,iWojr9R/XwhOONU4pj/Nrg0OZJihc4UvSnDqVhNePtmMbb...,2025-08-01 08:06:26+07:00,2025-08-01,IDN,-7.640300,111.516808,10.0,139.228.90.177,,0.0,0.0,B
2,WcP8SwCc9CeHO7OwyX6YxCzWvommp3FjNcniM+3DJlI2xV...,2025-08-01 19:30:45+07:00,2025-08-01,IDN,3.703892,98.658615,1.4,,,0.0,0.0,B
3,WcP8SwCc9CeHO7OwyX6YxCzWvommp3FjNcniM+3DJlI2xV...,2025-08-01 18:15:18+07:00,2025-08-01,IDN,3.695160,98.655952,3.9,,,0.0,0.0,B
4,WcP8SwCc9CeHO7OwyX6YxCzWvommp3FjNcniM+3DJlI2xV...,2025-07-31 09:01:18+07:00,2025-07-31,IDN,3.700020,98.672333,35.0,103.76.20.70,,0.0,0.0,B
...,...,...,...,...,...,...,...,...,...,...,...,...
2122337,I0OjaqIHs//5LMZpnWkNUubvplNVtq2lH/1LUr1cZsCaQ4...,2025-09-19 17:37:18+07:00,2025-09-19,MAR,34.939999,-4.320000,0.0,105.156.180.128,,0.0,0.0,E
2123639,/cnSksn4DW9UcKn6HfbiZJjsaw1DPye68PjlYddn2We6hZ...,2025-09-20 00:28:39+07:00,2025-09-19,MAR,31.630098,-7.987938,0.0,102.52.132.207,,0.0,0.0,E
2124127,swzq4NefCL4QZa2N9IMn/Um4w0aOeKOH/oS1eDlJ0yHNmi...,2025-09-20 02:37:18+07:00,2025-09-19,MAR,33.560000,-7.540000,0.0,105.74.2.105,,0.0,0.0,E
2124432,IZtmpg2N780kY1y9y+BGeMfP2dZ+UOTSKLlIrxMD8AmZC2...,2025-09-20 03:12:34+07:00,2025-09-19,MAR,33.610000,-7.630000,17.0,41.141.178.254,,0.0,0.0,E


In [48]:
cc=home_they[home_they.ad_id.isin(filtered_df.maid)]

In [45]:
home_they[home_they.ad_id.isin(filtered_df.maid)].geohash_5.value_counts()[:10]

geohash_5
eyk4m    439
everz    361
evcvn    309
eymbj    287
ev3jj    237
eyntk    220
ey56f    188
evfef    186
ey7gs    167
eyh3t    126
Name: count, dtype: int64

In [43]:
pd.read_parquet('/home/hieu/Work/new_casacom/blacklist_geohash.parquet').sort_values('gh7')

,gh7,total_records,lat,lon
0,dpz88gr,4.404385e+08,43.700180,-79.409866
1,ev0fhkq,2.001478e+09,28.500595,-9.999619
2,ev3hvxc,3.983004e+08,30.409470,-9.599991
3,ev3jj9w,1.667464e+09,30.419083,-9.593124
4,ev3jj9z,3.456402e+08,30.420456,-9.591751
5,ev3jjcb,3.442015e+08,30.420456,-9.590378
6,evcvnw1,3.452903e+08,33.256302,-8.501358
7,evd7cwp,1.029187e+09,31.630325,-8.009720
8,evd7cyn,6.014220e+08,31.630325,-8.000107
9,evd7cyu,2.265060e+09,31.634445,-8.002853


In [72]:
filtered_df.groupby('maid').size().sort_values(ascending=False)[:10]

maid
QI9DzMp32OFSUepqixiIN/45CmpiFQYL+1H9PcKW8BStzWaOUnh5jSuECFXirw41    23846
KS7KOAMEKWoxjgEMy6Id/s0ePmzqlilyfceVwi/zXe6w7IzbJLA9RbVvSpO8kBsh    20709
1WiokNr8d+4iOE+LFSLPmGABGzJkb0Ov/hxrdpgC0kLe1+N9VSwvMPbGFsUvoLQ+    18674
RQshaafesXOZqaBSIwxDnsslCDh9Oucs4Mpd9yKpBjd8Yt4lgIWeCjEuWWmTMmR8    16407
TcFjfpq4o6hQtLdVCFsk3oOwibKQGY6HyPhcEcAUD32x3ecG9I8XK4T8taWslx+u    16288
zoYUnKQzFPayZMPW+Fcpp0aRG9iZ3NzCjLdtb9FUoMIeT/aI6xzdB1nXtR+qG9s3    15509
k0HRm2QjR7QlXc3pJ98194ppMHSuJqSLFzLMKjGOHVP7shl7mRphW/towJoFxEHT    15475
FBWP0ww33XK8qOijcSJNubj+XQKgcxEuGvLP0g0nJJGeriFHNXbUQxdpkpK648il    15391
PfzR1AtKjfFHN5iAQdB5wUukud8BxJjivn09SbiOTRc8PQG0pAhhxN9bAA5hxT8a    14960
ZcyKf1jo5VCpRQtbjlymma5AJRndwAXq6GNPdUeosnypbld1zmvwOpOfBubG55AU    14601
dtype: int64

In [81]:
filtered_df.groupby('maid').size().sort_values(ascending=False)[50:170]

maid
82lpXSQ4Fy9KgMa9Qs2WGzoj7SrQjqzYzIgjudWEDSz02OhFyVmv2Y/9HIm2/uH2    7417
pecFjws4y77dFOwwXa9h/fwJEucC3KLpT3CWMtEO9uz6mhMnYqVWbzFRIyHQvlo7    7275
ALWSUddij6pHD3smnBP/mIsFH0FrwWRLAbsBMrtWs+58Aia3dqkVvZ8GyLviN3uo    7225
mvPZtbmShZbKnc4hb8NmmHAHUBl0e5Mz+0qx77WHrNDbFHWXQSoRD2m5LrM2iqni    7144
vmS9KWH2BuGCrDZANQag0bNrmsVrBHy48vdez8elJIxwvf5jmQ1NxjAVBeum8R1h    7046
                                                                    ... 
VjlxP+HdZjcvk1dzLG9ac9tLGCD6nN6iX99hdRdQL3wujn9aAmhGOIaaC8qcy6Vo    3072
TElLP2dMsQYp1kxjNBfWl0PsY9A4n+QTWmIGBSHobgP4yj6mpTEKRL0cOZYLwfAK    3059
OsPKiWaAXpPNwcwiIqWYoOH+Cr2pTCJ/B5R6lPQseSDcMWGv/0adg1ka+MUzBnYI    3036
MXhffGRht1YyHCr9W/BbZzdpK6dpIFLVO1EdKStvr87AxMQH65vZy28loE9WgVYf    3036
2rDIWwrI81JRaa/X5IjZImYtn8LGlt0oYIysHyRKI59BFs/4cK6D/p52+P1RGD3J    3033
Length: 120, dtype: int64

In [82]:
filtered_df.groupby('maid').size().sort_values(ascending=False)[170:190]

maid
OHXPkPAmWpqILaQnDZ14RPh+Lca428z21Ld9zwmAFykyYJBnONSs6PYuAgFxwCrf    3029
cQcc7/EPwIivr1hcF6Q7dTG3RXjZAa08VJtGPuIXV4P2xCDPVuPn4DRxiFSjyx9p    2999
ihEGMVwBo8eR2ViFX56VSCW45r+VTOJ5/mXeMrk2Db3M0CfuWfecHVsQQQHBJyoo    2991
eWxy1yEh9MTt7WLfO5A9RJ3Gjaw1SITSbG5EF15eNvQnLmqyQ8KJqY8ykjht7sQH    2979
cKQYZ/XCK783dJTNMRk4oSanZiuAYG5jh2lexyJbIk9GUUcvCBLpH5sK1iBBB5CN    2972
N6pm9Zjcb5JAHFzznyrq6S3aNkDaOYz05fkIj/Ny/IEgf4Pdh1fBkyunrzpT8y4m    2955
65vr2e9mXTnzp2h/KEoHIywRolBeTwd5ZUnWlYd1UMCian1dPbf4DtEg/05Z5FYh    2939
cn7XavA+8jeJHx12MLCIyc/bm4OL2iuTKdYx9lCCjnRI1nPr7AwNl1AQklUgsp4Y    2934
w4XdujvsYbqXwfaGRyrLn0vqrFoWCkzoCzw/bKYSv0MOff59JHkP4ghi0veO5PIg    2857
FPYpJabi2yjcgDkrJKCuUUYYPasLpKjwCnqxwGiscI+W3g6hAiu99te+XJRkrapz    2836
tZhOXSjP8VwEVP3497pHH+7amFd5d8VcIaLfNwrk5uDcYO7vjpgK4O/eSRnzZCNM    2782
IK2YVQO/qW/6PpIflqeNOL49PQ9Gl4K3Z8FmVL7vjQ2WK/LsIYGOl1AYv6fLo10Z    2766
TN1cVNk5yK7CQONJtw52ufsLm0+r6lBC9NDBUj1zhsfQDdD86QUP9J/xmpP76v+4    2752
0N8j8azyVKN4vMvPYEkbhPzu6G7e/XvGok6HVxgLj9FyoR

In [85]:
filtered_df.groupby('maid').size().sort_values(ascending=False)[500:520]

maid
5JaHAsZ57aH62czzQeNkCLVBhilszi9ENwWHZJy+8rkrANnXXdS8D7Typu/d0yMj    843
+dnSkskIz3FIYLT+KDfBXxQ65DjKJRERIACk13V/6vZje/JIkBbphd3/GixS+HKY    841
3eSgZoBSJe67atpoa4JGJmTcCuN4DuHewUcP6bOiwPFkDL3M1KjgdTLMdhjblGeL    839
SfR+QaeJ7V63Fo+cA0iWlx4VPMwWx/lcfJq3j6ebcUuZcW5YGEYq/h7DTn2JKsBN    838
Rw0P4rjSN8vJTMnHV5naYMR8UjMgu+Q+kF9D/K9qbkSGVwRifY9nhCXD01TTbVyq    836
U7Zwv6zHlS13zytSf6sFNE4mp8LxLTk2YClpGS2Fd+7ABfKtgoo0aPUsMH8Yh2D8    830
M0IBCYFOmiomtc1sR7XMPQIIBsqb0n0Eth0yB7L99zvW/G2puK7lgC2hyaDDXTh3    826
kZ+jlZzB9+UzyQjn6j8TmK1yaEyyxeABe0NWvM+eIi9JoLq4LZRiFGboI2/S/rHo    826
CVcslEoV72QV+yTrGonBfUBxTELXb2aPfBkXkU79/RF5tFypCH/SjH0HJ+0j7GKT    825
rAGjH22B3kwJu9G/JnE3wdDINXnv/XLjZAlo+YLLsogih+bH9ZFfBFhm2ZRk41YD    822
ttNFAT34ECooKd1XJbQ8lkULvK//zkGrYXhZPxfDjcAXZpO8am7jBkT7WmyttnaX    818
rnyCE2wb/x9/Z7k1pCR1AsqMiayTM3CJAqx74tIfDFk7re8qWmBIS/3IM8+m+GOl    815
iEcyK1zQKjiv851WDWiMwY2N6MGxrQrYq+b9yMRWA7t7BPcnX/NgAKCbnKdnyU/g    812
YQRnDczDrVMkreUaZ9SUg699C0YbxHpMa4EJhTPOQBJ/IX/i2BcEtog1Mym

In [86]:
filtered_df.groupby('maid').size().sort_values(ascending=False)[800:820]

maid
dTRbvMxCVMQFd51k7WLX1aTd1qVGEZJK6x/xVtzvZWczz/DiavFpT8kIROB9L6ye    390
taE8ewbr16fxutyLZP3sSH9zv0mPoXv/g8Tf5UyHaKXbaj3KCQdM+S2smykwmguj    389
AHlPJdEzqtJ3xrXvNH0IYFSZRL5CTt4SUQH0oYKnXfVJ8Sdytk001TJTpEzI5n1x    389
/oU5wkFaGoMLYjeAwO1rknwekBT3ye7oB7/8Nf7szJ3YhxL8ZCjE/Tm1Yo9+v/g1    388
WyPzvbOy5N2fMldvOchepmEx3JsUmfg+zhVW0Vm3DWVr4O1PCdljr14f81QXe7/9    387
5hob+7o7jbc9Fjx66A8+uUBvZn47S0RnoiWp1QA3AL9ucZGj3Zn7xu6C+gNQVr/9    386
EqfGYQwX0dSMjpOMqurGyNOXUGGF2YuiL9t6nJ/YM2KDahgJy5vUW8uKCPuYchfh    386
sPM+fEgWP+NBEQUaGQ7x43xOzdCZsOcIuyXb5L58WHeK4DIrMXy90YNrZgW4a/mV    386
J5nEGQSQqoOuxHH443OIPpYDIt+bF6sCG8fIfsmpbGNsvW31RzMAGYt91q6dBxKM    386
nvDQh7zODPss2mmu61ixXqmXIlUyj/oCHChhHnxqn6c6o2jk7FHSG1RbkFPLnEhT    386
6/KQmFtSYsAsXVCchIWEaEEOzNMZiuaWlafH4kTGPZq2HfdyXUfIT6B9cuka1ioW    386
Dy3ZmxpHLiFXQKpePputT/A+XWCrEAXkWjyjqt4wisju6cXtnhV+hfJRHL3zgeYx    385
oIoTJ5zHrEeQMDvs2kovzcolp8frs0m+MHgBZxDpAOvtGFPFPV/nzxcmgOxIt893    384
nfXkeRSisGgek14pbXC2lSNoSOOMPQ7+ujk2RZZlbnR/kX8iJ4QLuXvPb0+

In [118]:
import pygeohash as pgh
import plotly.express as px

# Prepare a DataFrame for home_detected points
import pandas as pd

def plot_home_detected(maid):
    # Filter the sample dataframe for this maid
    sample_df = filtered_df[filtered_df.maid == maid]
    home_detected = [pgh.decode(gh) for gh in set(cc[cc.ad_id == maid].geohash_5.to_list())]
    home_detected_df = pd.DataFrame(home_detected, columns=['latitude', 'longitude'])
    # Decode geohash_5 to (lat, lon) for detected home locations
    
    # Plot all sample_df points and overlay home_detected points using Plotly with OSM map
    fig = px.scatter_mapbox(
        sample_df,
        lat="latitude",
        lon="longitude",
        zoom=10,
        height=600,
        mapbox_style="open-street-map",
        title=f"Sample Points and Detected Home Locations {sample_df.shape} maid {maid}"
    )

    # Add home_detected points as a separate trace (in red)
    fig.add_scattermapbox(
        lat=home_detected_df['latitude'],
        lon=home_detected_df['longitude'],
        mode='markers',
        marker=dict(size=14, color='red'),
        name='Detected Home'
    )

    # Show the plot
    fig.show()
    fig.write_html(f'{maid.replace("/","_")}.html')
    


In [119]:
list_maid=['QI9DzMp32OFSUepqixiIN/45CmpiFQYL+1H9PcKW8BStzWaOUnh5jSuECFXirw41','OHXPkPAmWpqILaQnDZ14RPh+Lca428z21Ld9zwmAFykyYJBnONSs6PYuAgFxwCrf','82lpXSQ4Fy9KgMa9Qs2WGzoj7SrQjqzYzIgjudWEDSz02OhFyVmv2Y/9HIm2/uH2','tbr42AW59VdnVYImxV8kIdTPNot6EJSc0m+6voBad3xq/9jMPfhlAm24+CY+cex4','DcZLrWw2jkWoS3uoktCPFeI6h2tXkJi3ABCxx89re6dKz9Ccl1gaQxegxCH/3w2/','avK0mGOGN/qEqgFkoHLdXtT71u9raH9+Hm90oMjQsdyitDIj9mOyt8qAqCsJdTcp']
for i in list_maid:
    plot_home_detected(i)

/tmp/ipykernel_5265/3279441594.py:15: DeprecationWarning:

*scatter_mapbox* is deprecated! Use *scatter_map* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/



/tmp/ipykernel_5265/3279441594.py:15: DeprecationWarning:

*scatter_mapbox* is deprecated! Use *scatter_map* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/



/tmp/ipykernel_5265/3279441594.py:15: DeprecationWarning:

*scatter_mapbox* is deprecated! Use *scatter_map* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/



/tmp/ipykernel_5265/3279441594.py:15: DeprecationWarning:

*scatter_mapbox* is deprecated! Use *scatter_map* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/



/tmp/ipykernel_5265/3279441594.py:15: DeprecationWarning:

*scatter_mapbox* is deprecated! Use *scatter_map* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/



/tmp/ipykernel_5265/3279441594.py:15: DeprecationWarning:

*scatter_mapbox* is deprecated! Use *scatter_map* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/



In [100]:
pd.set_option('display.max_columns', None)

In [126]:
filtered_df[filtered_df.maid==list_maid[1]]

,maid,timestamp,date,country,latitude,longitude,horizontal_accuracy,ipv4,ipv6,altitude,altitude_accuracy,flux
50743,OHXPkPAmWpqILaQnDZ14RPh+Lca428z21Ld9zwmAFykyYJ...,2025-09-06 19:50:18+07:00,2025-09-06,FRA,48.549779,-1.924873,22.0,2a04:cec0:1024:2ad8:5573:6a06:6f7b:886a,,18.0,4.0,F
50744,OHXPkPAmWpqILaQnDZ14RPh+Lca428z21Ld9zwmAFykyYJ...,2025-09-06 12:24:49+07:00,2025-09-06,FRA,48.596596,-1.964519,2.0,2a04:cec0:1024:2ad8:5573:6a06:6f7b:886a,,27.0,3.0,F
50745,OHXPkPAmWpqILaQnDZ14RPh+Lca428z21Ld9zwmAFykyYJ...,2025-09-06 12:37:26+07:00,2025-09-06,FRA,48.631621,-2.058894,4.0,2a04:cec0:1024:2ad8:5573:6a06:6f7b:886a,,13.0,3.0,F
50746,OHXPkPAmWpqILaQnDZ14RPh+Lca428z21Ld9zwmAFykyYJ...,2025-09-06 12:30:22+07:00,2025-09-06,FRA,48.613620,-2.042310,2.0,2a04:cec0:1024:2ad8:5573:6a06:6f7b:886a,,47.0,3.0,F
50747,OHXPkPAmWpqILaQnDZ14RPh+Lca428z21Ld9zwmAFykyYJ...,2025-09-06 12:32:15+07:00,2025-09-06,FRA,48.620839,-2.057947,2.0,2a04:cec0:1024:2ad8:5573:6a06:6f7b:886a,,37.0,3.0,F
...,...,...,...,...,...,...,...,...,...,...,...,...
1964236,OHXPkPAmWpqILaQnDZ14RPh+Lca428z21Ld9zwmAFykyYJ...,2025-09-10 00:35:09+07:00,2025-09-09,FRA,48.527176,-1.771426,3.0,2a04:cec0:102d:5daf:5d45:853:630c:6e58,,37.0,3.0,F
1964237,OHXPkPAmWpqILaQnDZ14RPh+Lca428z21Ld9zwmAFykyYJ...,2025-09-09 20:35:21+07:00,2025-09-09,FRA,48.535722,-1.779457,30.0,2a04:cec0:102d:5daf:5d45:853:630c:6e58,,19.0,3.0,F
1964238,OHXPkPAmWpqILaQnDZ14RPh+Lca428z21Ld9zwmAFykyYJ...,2025-09-09 20:32:15+07:00,2025-09-09,FRA,48.543655,-1.813786,2.0,2a04:cec0:102d:5daf:5d45:853:630c:6e58,,22.0,3.0,F
1964239,OHXPkPAmWpqILaQnDZ14RPh+Lca428z21Ld9zwmAFykyYJ...,2025-09-09 13:13:57+07:00,2025-09-09,FRA,48.619092,-2.057565,4.0,2a04:cec0:102d:5daf:5d45:853:630c:6e58,,39.0,3.0,F


In [111]:
leisure[leisure.maid=='82lpXSQ4Fy9KgMa9Qs2WGzoj7SrQjqzYzIgjudWEDSz02OhFyVmv2Y/9HIm2/uH2']

,maid,geohash,category,confidence,country,lat,lon,first_seen,last_seen,est_duration,pings,unique_days,evidence_score,fluxB,fluxC,fluxD,fluxE,fluxF,monday,tuesday,wednesday,thursday,friday,saturday,sunday,month_1,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12,hour_0,hour_1,hour_2,hour_3,hour_4,hour_5,hour_6,hour_7,hour_8,hour_9,hour_10,hour_11,hour_12,hour_13,hour_14,hour_15,hour_16,hour_17,hour_18,hour_19,hour_20,hour_21,hour_22,hour_23,poi_score,poi_info,home,work,leisure,path,maid_total_pings,cell_info,maid_fluxB,maid_fluxC,maid_fluxD,maid_fluxE,maid_fluxF,maid_countries,maid_countries_pings,maid_first_seen,maid_last_seen
67013,82lpXSQ4Fy9KgMa9Qs2WGzoj7SrQjqzYzIgjudWEDSz02O...,f256734,leisure,0.480671,Canada,45.401674,-74.033551,2025-03-22T23:19:55-04:00,2025-09-15T20:10:06-04:00,885,10142,72,"{'home': 0.362036672445414, 'work': 0.29864814...",6564,3042,71,67,398,8,8,8,13,10,14,11,0,0,4,10,9,0,3,31,15,0,0,0,34,4,6,0,0,0,650,73,1293,19,1456,1458,13,14,224,169,142,140,6,108,2826,1291,13,203,NaN,NaN,0.362037,0.298648,0.480671,0.0,24440,"{'f256734': {'distance': 0, 'radio': 'unknown'...",11522,5848,144,238,629,"{'Canada', 'Morocco'}","{'Canada': 18359, 'Morocco': 22}",2025-03-18T20:22:00-04:00,2025-09-15T20:10:06-04:00
67014,82lpXSQ4Fy9KgMa9Qs2WGzoj7SrQjqzYzIgjudWEDSz02O...,f2566g6,leisure,0.235439,Canada,45.414627,-74.055039,2025-03-18T20:22:00-04:00,2025-08-11T21:54:58-04:00,911,2144,50,"{'home': 0.1935506053681209, 'work': 0.1751521...",1099,893,10,142,0,8,8,6,8,8,5,7,0,0,3,14,26,6,0,1,0,0,0,0,10,7,1,2,0,10,310,29,54,51,43,29,187,34,55,429,73,97,73,166,174,125,112,73,NaN,NaN,0.193551,0.175152,0.235439,0.0,24440,"{'f2566g6': {'distance': 0, 'radio': 'unknown'...",11522,5848,144,238,629,"{'Canada', 'Morocco'}","{'Canada': 18359, 'Morocco': 22}",2025-03-18T20:22:00-04:00,2025-09-15T20:10:06-04:00


In [132]:
work[work.maid=='OHXPkPAmWpqILaQnDZ14RPh+Lca428z21Ld9zwmAFykyYJBnONSs6PYuAgFxwCrf'].sort_values('confidence',ascending=False).to_parquet('detected_wrong.parquet')

In [113]:
home[home.maid=='82lpXSQ4Fy9KgMa9Qs2WGzoj7SrQjqzYzIgjudWEDSz02OhFyVmv2Y/9HIm2/uH2']

,maid,geohash,category,confidence,country,lat,lon,first_seen,last_seen,est_duration,pings,unique_days,evidence_score,fluxB,fluxC,fluxD,fluxE,fluxF,monday,tuesday,wednesday,thursday,friday,saturday,sunday,month_1,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12,hour_0,hour_1,hour_2,hour_3,hour_4,hour_5,hour_6,hour_7,hour_8,hour_9,hour_10,hour_11,hour_12,hour_13,hour_14,hour_15,hour_16,hour_17,hour_18,hour_19,hour_20,hour_21,hour_22,hour_23,poi_score,poi_info,home,work,leisure,path,maid_total_pings,cell_info,maid_fluxB,maid_fluxC,maid_fluxD,maid_fluxE,maid_fluxF,maid_countries,maid_countries_pings,maid_first_seen,maid_last_seen


In [106]:
import duckdb 
work_they=duckdb.query("""
select * from read_parquet('/home/hieu/Work/casacom/import/work/2025/**/*.parquet')
where ad_id = 'OHXPkPAmWpqILaQnDZ14RPh+Lca428z21Ld9zwmAFykyYJBnONSs6PYuAgFxwCrf'
""").df()



FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [130]:
home_they[home_they.ad_id=='OHXPkPAmWpqILaQnDZ14RPh+Lca428z21Ld9zwmAFykyYJBnONSs6PYuAgFxwCrf']

,ad_id,id_type,geohash_5,country,iso_country_code,region,city,zipcode,census_block_group
3778,OHXPkPAmWpqILaQnDZ14RPh+Lca428z21Ld9zwmAFykyYJ...,idfa,gbwsp,france,FR,brittany,baguer-morvan,,NA
7321,OHXPkPAmWpqILaQnDZ14RPh+Lca428z21Ld9zwmAFykyYJ...,idfa,gbwsp,france,FR,brittany,baguer-morvan,,NA


In [107]:
work_they

,ad_id,id_type,geohash_5,country,iso_country_code,region,city,zipcode,census_block_group
0,OHXPkPAmWpqILaQnDZ14RPh+Lca428z21Ld9zwmAFykyYJ...,idfa,gbwsp,france,FR,brittany,baguer-morvan,,NA
1,OHXPkPAmWpqILaQnDZ14RPh+Lca428z21Ld9zwmAFykyYJ...,idfa,gbwsp,france,FR,brittany,baguer-morvan,,NA


In [1]:
import duckdb

conn = duckdb.connect()
conn.execute("SET timezone='UTC'")
df = conn.query("""
SELECT 
    *
FROM read_parquet('/home/hieu/Work/new_casacom/data/raw_rt/**/*.parquet')
where maid ='QI9DzMp32OFSUepqixiIN/45CmpiFQYL+1H9PcKW8BStzWaOUnh5jSuECFXirw41'
""").df()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [6]:
df.sort_values('timestamp',ascending=True,inplace=True)
df.reset_index(drop=True,inplace=True)

In [49]:
import plotly.express as px

# Filter the DataFrame for the specific date
df_plot = df[df.date == '2025-08-01'].copy()

# Add an 'hour' column if not already present
if 'hour' not in df_plot.columns:
    # Assuming 'timestamp' is a datetime64[ns, UTC] column
    df_plot['hour'] = df_plot['timestamp'].dt.hour

# Create a color column based on hour to highlight it
df_plot['hour_str'] = df_plot['hour'].astype(str)  # For better legend display

# Plot with color highlighting the hour
fig = px.scatter_mapbox(
    df_plot,
    lat="latitude",
    lon="longitude",
    color="hour_str",  # Highlight by hour
    hover_data=df_plot.columns,  # Show all columns on hover
    zoom=10,
    height=500
)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

# Show the plot
fig.show()

/tmp/ipykernel_4782/2127055434.py:15: DeprecationWarning:

*scatter_mapbox* is deprecated! Use *scatter_map* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/



In [50]:
df_plot[df_plot['hour_str']=='8'].sort_values('timestamp',ascending=True)

,maid,timestamp,date,country,latitude,longitude,horizontal_accuracy,ipv4,ipv6,altitude,altitude_accuracy,flux,geohash,hour,hour_str
300,QI9DzMp32OFSUepqixiIN/45CmpiFQYL+1H9PcKW8BStzW...,2025-08-01 08:00:42+00:00,2025-08-01,MAR,30.420099,-9.59124,9.0,197.146.103.143,,0.0,0.0,B,ev3jj9z,8,8
301,QI9DzMp32OFSUepqixiIN/45CmpiFQYL+1H9PcKW8BStzW...,2025-08-01 08:01:37+00:00,2025-08-01,MAR,30.420139,-9.59120,9.0,197.146.103.143,,0.0,0.0,B,ev3jj9z,8,8
302,QI9DzMp32OFSUepqixiIN/45CmpiFQYL+1H9PcKW8BStzW...,2025-08-01 08:01:44+00:00,2025-08-01,MAR,30.420071,-9.59124,9.0,197.146.103.143,,0.0,0.0,B,ev3jj9z,8,8
303,QI9DzMp32OFSUepqixiIN/45CmpiFQYL+1H9PcKW8BStzW...,2025-08-01 08:02:06+00:00,2025-08-01,MAR,30.420090,-9.59125,9.0,197.146.103.143,,0.0,0.0,B,ev3jj9z,8,8
304,QI9DzMp32OFSUepqixiIN/45CmpiFQYL+1H9PcKW8BStzW...,2025-08-01 08:04:51+00:00,2025-08-01,MAR,28.500031,-9.99997,6.0,105.74.67.2,,0.0,0.0,B,ev0fhkq,8,8
305,QI9DzMp32OFSUepqixiIN/45CmpiFQYL+1H9PcKW8BStzW...,2025-08-01 08:05:30+00:00,2025-08-01,MAR,28.499990,-9.99996,21.0,105.74.67.2,,0.0,0.0,B,ev0fhkq,8,8
306,QI9DzMp32OFSUepqixiIN/45CmpiFQYL+1H9PcKW8BStzW...,2025-08-01 08:05:49+00:00,2025-08-01,MAR,28.500019,-10.00003,8.0,105.74.67.2,,0.0,0.0,B,ev0fhkq,8,8
307,QI9DzMp32OFSUepqixiIN/45CmpiFQYL+1H9PcKW8BStzW...,2025-08-01 08:06:15+00:00,2025-08-01,MAR,28.500031,-10.00003,28.0,105.74.67.2,,0.0,0.0,B,ev0fhkq,8,8
308,QI9DzMp32OFSUepqixiIN/45CmpiFQYL+1H9PcKW8BStzW...,2025-08-01 08:06:29+00:00,2025-08-01,MAR,28.499990,-9.99996,29.0,105.74.67.2,,0.0,0.0,B,ev0fhkq,8,8
309,QI9DzMp32OFSUepqixiIN/45CmpiFQYL+1H9PcKW8BStzW...,2025-08-01 08:06:52+00:00,2025-08-01,MAR,28.499981,-9.99999,18.0,105.74.67.2,,0.0,0.0,B,ev0fhkq,8,8


In [45]:
pgh.geohash_haversine_distance('ev0fhkq','evd7cyu')/1000

397.9113006759899

In [27]:
# evidence_store_no_poi.py
import datetime as dt
import math
import pickle
import gzip
import os
import time
import tempfile
from pathlib import Path
from typing import Dict, List, Union, Optional, Tuple
import pandas as pd
import duckdb
import pygeohash as pgh
from collections import defaultdict
import json

conn=duckdb.connect()
conn.execute("SET timezone='UTC'")

def _to_dt(x: Union[str, dt.datetime]) -> dt.datetime:
    """Convert input to datetime object, ensuring UTC timezone consistency"""
    if isinstance(x, str):
        # Parse string and assume it's in UTC if no timezone info
        dt_obj = dt.datetime.fromisoformat(x.replace('Z', '+00:00'))
        if dt_obj.tzinfo is None:
            dt_obj = dt_obj.replace(tzinfo=dt.timezone.utc)
        return dt_obj
    elif isinstance(x, dt.datetime):
        # Ensure datetime has UTC timezone
        if x.tzinfo is None:
            return x.replace(tzinfo=dt.timezone.utc)
        return x
    else:
        raise ValueError(f"Unsupported type for datetime conversion: {type(x)}")

class EvidenceStore:
    def __init__(self):
        self.store: Dict[str, dict] = {}

    def _init(self):
        return {
            'pings': 0,
            'first_seen_ts': None,
            'last_seen_ts': None,
            'unique_days': set(),
            'hourly_hist': [0]*24,
            'weekday_hist': [0]*7,
            'hourly_hist_weekday': [0]*24,
            'hourly_hist_weekend': [0]*24,
            'monthly_hist': {},
            'daily_flags': {},
            'max_seen_date': None,
            'gap_bins': {'0d':0, '1-3d':0, '4-7d':0, '8-30d':0, '>30d':0},
            'poi_info': None,
            'poi_calculated': False,
            'mean_lat': None,
            'mean_lon': None,
            'coord_count': 0,
            # Simplified duration tracking
            'hourly_minutes': {}, # Format: {hour: {'min': min_minute, 'max': max_minute}}
            'est_duration': 0,
            # Flux tracking
            'flux_counts': {'B': 0, 'C': 0, 'D': 0, 'E': 0, 'F': 0},
        }
    
    def _update_mean_coordinates(self, c: dict, lat: float, lon: float):
        """Update running mean of coordinates using incremental formula"""
        if c['mean_lat'] is None or c['mean_lon'] is None:
            c['mean_lat'] = lat
            c['mean_lon'] = lon
            c['coord_count'] = 1
        else:
            n = c['coord_count']
            c['mean_lat'] = (c['mean_lat'] * n + lat) / (n + 1)
            c['mean_lon'] = (c['mean_lon'] * n + lon) / (n + 1)
            c['coord_count'] += 1

    @staticmethod
    def _shrink_ratio(ratio: float, n: int, p0: float, a: float = 8.0) -> float:
        return (ratio * n + a * p0) / (n + a) if (n + a) > 0 else p0

    @staticmethod
    def _mask_for(ts: dt.datetime) -> int:
        h = ts.hour
        wd = ts.weekday()
        mask = 0
        if 4 <= h <= 6:
            mask |= 0x1
        if 20 <= h <= 23:
            mask |= 0x2
        if (9 <= h <= 17) and (wd < 5):
            mask |= 0x4
        if (h >= 22) or (h <= 5):
            mask |= 0x8
        return mask

    def update(self, new_data: Dict[str, List[Union[str, dt.datetime]]], 
               coordinates: Optional[Dict[str, Tuple[float, float]]] = None,
               flux_data: Optional[Dict[str, List[str]]] = None):
        """Update evidence store with timestamp data and simplified duration calculation"""
        for gh, ts_list in (new_data or {}).items():
            if not ts_list: 
                continue
            ts_objs = [_to_dt(x) for x in ts_list]
            ts_objs.sort()
            c = self.store.get(gh)
            if c is None:
                c = self._init()
                self.store[gh] = c

            # Update mean coordinates if provided
            if coordinates and gh in coordinates:
                lat, lon = coordinates[gh]
                self._update_mean_coordinates(c, lat, lon)

            # Update flux counts if provided
            if flux_data and gh in flux_data:
                for flux_type in flux_data[gh]:
                    if flux_type in c['flux_counts']:
                        c['flux_counts'][flux_type] += 1

            # Initialize or reset hourly_minutes for this update
            hourly_minutes = {}
            
            for ts in ts_objs:
                d = ts.date()
                h = ts.hour
                minute = ts.minute
                wd = ts.weekday()
                
                # Track min/max minute for each hour
                if h not in hourly_minutes:
                    hourly_minutes[h] = {'min': minute, 'max': minute}
                else:
                    hourly_minutes[h]['min'] = min(hourly_minutes[h]['min'], minute)
                    hourly_minutes[h]['max'] = max(hourly_minutes[h]['max'], minute)
                
                # Update other tracking fields
                c['first_seen_ts'] = ts if c['first_seen_ts'] is None or ts < c['first_seen_ts'] else c['first_seen_ts']
                c['last_seen_ts']  = ts if c['last_seen_ts']  is None or ts > c['last_seen_ts']  else c['last_seen_ts']
                c['pings'] += 1
                c['unique_days'].add(d)
                c['hourly_hist'][h] += 1
                c['weekday_hist'][wd] += 1
                if wd >= 5:
                    c['hourly_hist_weekend'][h] += 1
                else:
                    c['hourly_hist_weekday'][h] += 1
                mkey = f"{ts.year:04d}-{ts.month:02d}"
                c['monthly_hist'][mkey] = c['monthly_hist'].get(mkey, 0) + 1
                ord_key = d.toordinal()
                c['daily_flags'][ord_key] = c['daily_flags'].get(ord_key, 0) | self._mask_for(ts)
                if c['max_seen_date'] is None or d > c['max_seen_date']:
                    if c['max_seen_date'] is not None:
                        delta = (d - c['max_seen_date']).days
                        if delta == 0:
                            c['gap_bins']['0d'] += 1
                        elif 1 <= delta <= 3:
                            c['gap_bins']['1-3d'] += 1
                        elif 4 <= delta <= 7:
                            c['gap_bins']['4-7d'] += 1
                        elif 8 <= delta <= 30:
                            c['gap_bins']['8-30d'] += 1
                        else:
                            c['gap_bins']['>30d'] += 1
                    c['max_seen_date'] = d
            
            # Merge hourly_minutes from this update into stored data
            if not c.get('hourly_minutes'):
                c['hourly_minutes'] = {}
                
            for hour, minmax in hourly_minutes.items():
                if hour not in c['hourly_minutes']:
                    c['hourly_minutes'][hour] = minmax
                else:
                    c['hourly_minutes'][hour]['min'] = min(c['hourly_minutes'][hour]['min'], minmax['min'])
                    c['hourly_minutes'][hour]['max'] = max(c['hourly_minutes'][hour]['max'], minmax['max'])
            
            # Always recalculate estimated duration from the complete hourly_minutes
            # This ensures consistency regardless of batch size
            total_duration = 0
            for hour, minmax in c['hourly_minutes'].items():
                duration = minmax['max'] - minmax['min']
                total_duration += duration
            
            c['est_duration'] = total_duration

    def save_to_pickle(self, filepath: str, compress: bool = False) -> None:
        """Save EvidenceStore to pickle format with atomic write
        
        Args:
            filepath: Path to save the pickle file
            compress: Whether to use gzip compression (default: False for <1MB files)
        """
        try:
            # Prepare data for serialization
            data = {
                'store': self.store,
                'version': '1.0',  # For future compatibility
                'created_at': dt.datetime.now().isoformat()
            }
            
            # Ensure directory exists
            Path(filepath).parent.mkdir(parents=True, exist_ok=True)
            
            # Determine final filepath with proper extension
            if compress:
                if not filepath.endswith('.gz'):
                    filepath += '.gz'
            else:
                if not any(filepath.endswith(ext) for ext in ['.pkl', '.pickle']):
                    filepath += '.pkl'
            
            # Atomic write: write to temp file first, then atomically replace
            temp_dir = os.path.dirname(filepath)
            with tempfile.NamedTemporaryFile(mode='wb', dir=temp_dir, delete=False, suffix='.tmp') as temp_file:
                temp_path = temp_file.name
                
                try:
                    if compress:
                        with gzip.open(temp_file, 'wb', compresslevel=9) as f:
                            pickle.dump(data, f, protocol=pickle.HIGHEST_PROTOCOL)
                    else:
                        pickle.dump(data, temp_file, protocol=pickle.HIGHEST_PROTOCOL)
                    
                    # Ensure data is written to disk
                    temp_file.flush()
                    os.fsync(temp_file.fileno())
                    
                except Exception:
                    # Clean up temp file on error
                    try:
                        os.unlink(temp_path)
                    except OSError:
                        pass
                    raise
                
                # Atomically replace target file
                os.replace(temp_path, filepath)
                    
        except Exception:
            raise
    
    def load_from_pickle(self, filepath: str, max_retries: int = 3) -> None:
        """Load EvidenceStore from pickle format with retry on transient errors
        
        Args:
            filepath: Path to the pickle file
            max_retries: Maximum number of retry attempts for transient errors
        """
        last_exception = None
        
        for attempt in range(max_retries + 1):
            try:
                # Auto-detect compression based on file extension
                if filepath.endswith('.gz'):
                    with gzip.open(filepath, 'rb') as f:
                        data = pickle.load(f)
                elif any(filepath.endswith(ext) for ext in ['.pkl', '.pickle']):
                    with open(filepath, 'rb') as f:
                        data = pickle.load(f)
                else:
                    # Try uncompressed first, fallback to compressed
                    try:
                        with open(filepath, 'rb') as f:
                            data = pickle.load(f)
                    except (pickle.UnpicklingError, UnicodeDecodeError):
                        with gzip.open(filepath, 'rb') as f:
                            data = pickle.load(f)
                
                # Load store data
                if 'store' in data:
                    self.store = data['store']
                else:
                    # Legacy format support (direct store)
                    self.store = data
                    
                # Initialize flux_counts for older data formats
                for gh, c in self.store.items():
                    if 'flux_counts' not in c:
                        c['flux_counts'] = {'B': 0, 'C': 0, 'D': 0, 'E': 0, 'F': 0}
                
                # Success - return
                return
                
            except FileNotFoundError:
                print(f"File not found: {filepath}")
                raise
            except (EOFError, pickle.UnpicklingError, OSError) as e:
                last_exception = e
                if attempt < max_retries:
                    # Transient error - retry with exponential backoff
                    sleep_time = 0.1 * (2 ** attempt)  # 0.1s, 0.2s, 0.4s
                    print(f"Transient error loading {filepath} (attempt {attempt + 1}/{max_retries + 1}): {e}. Retrying in {sleep_time}s...")
                    time.sleep(sleep_time)
                    continue
                else:
                    # Max retries exceeded
                    print(f"Error loading EvidenceStore from {filepath} after {max_retries + 1} attempts: {e}")
                    raise
            except Exception as e:
                # Non-transient error - don't retry
                print(f"Error loading EvidenceStore from {filepath}: {e}")
                raise
        
        # Should not reach here, but just in case
        if last_exception:
            raise last_exception
    
    def save(self, filepath: str, compress: bool = False) -> None:
        """Convenience method to save the store
        
        Args:
            filepath: Path to save the file
            compress: Whether to use compression (default: False for fast I/O)
        """
        self.save_to_pickle(filepath, compress=compress)
    
    def load(self, filepath: str) -> None:
        """Convenience method to load the store
        
        Args:
            filepath: Path to load from
        """
        self.load_from_pickle(filepath)
    
    def clear_store(self) -> None:
        """Clear all evidence data"""
        self.store.clear()
        
    def recalculate_durations(self) -> None:
        """Recalculate all duration estimates in the store
        
        This can be useful when loading older data formats or ensuring
        consistency across different processing batches.
        """
        for gh, data in self.store.items():
            if 'hourly_minutes' in data:
                total_duration = 0
                for hour, minmax in data['hourly_minutes'].items():
                    duration = minmax['max'] - minmax['min']
                    total_duration += duration
                data['est_duration'] = total_duration

    def derive(self, gh: str) -> dict:
        if gh not in self.store: 
            return None
        c = self.store[gh]
        visits = c['pings']
        if visits == 0:
            return None

        first, last = c['first_seen_ts'], c['last_seen_ts']
        # Convert string timestamps to datetime objects if needed
        if isinstance(first, str):
            first = _to_dt(first)
        if isinstance(last, str):
            last = _to_dt(last)
        
        span_days = (last.date() - first.date()).days + 1 if first and last else 0
        unique_days = len(c['unique_days'])
        
        # Improved active_day_ratio with span capping and continuity boost
        capped_span = max(30, span_days)
        base_active_ratio = min(1.0, unique_days / capped_span) if capped_span > 0 else 0.0
        
        # Calculate continuity boost from gap patterns
        total_gaps = max(1, sum(c['gap_bins'].values()))
        continuity = (c['gap_bins'].get('0d', 0) + c['gap_bins'].get('1-3d', 0)) / total_gaps
        active_day_ratio = base_active_ratio * (0.5 + 0.5 * continuity)

        # visit-based ratios
        night_idxs = [22, 23, 0, 1, 2, 3, 4, 5]
        night_ratio = sum(c['hourly_hist'][i] for i in night_idxs) / visits
        weekday_day_ratio = sum(c['hourly_hist_weekday'][i] for i in range(9, 18)) / visits
        weekend_ratio = (c['weekday_hist'][5] + c['weekday_hist'][6]) / visits
        midday_weekday_ratio = sum(c['hourly_hist_weekday'][i] for i in range(11, 15)) / visits
        
        # Add evening_ratio for leisure scoring
        evening_ratio = sum(c['hourly_hist'][i] for i in range(18, 22)) / visits

        # day-level ratios from flags (presence-only)
        flags = list(c['daily_flags'].values())
        days_night = sum(1 for m in flags if (m & 0x8))
        days_weekday_work = sum(1 for m in flags if (m & 0x4))
        days_late_evening = sum(1 for m in flags if (m & 0x2))
        days_early_morning = sum(1 for m in flags if (m & 0x1))
        if unique_days > 0:
            night_days_ratio = days_night / unique_days
            weekday_work_days_ratio = days_weekday_work / unique_days
            late_evening_days_ratio = days_late_evening / unique_days
            early_morning_days_ratio = days_early_morning / unique_days
        else:
            night_days_ratio = weekday_work_days_ratio = late_evening_days_ratio = early_morning_days_ratio = 0.0

        # entropy of hourly distribution (normalized)
        H = 0.0
        for cnt in c['hourly_hist']:
            if cnt > 0:
                p = cnt / visits
                H -= p * math.log(p + 1e-12)
        entropy_hour_norm = H / math.log(24)

        # monthly stability via CV -> 1/(1+CV)
        months = list(c['monthly_hist'].values())
        if len(months) >= 2:
            mean_m = sum(months) / len(months)
            var_m = sum((x - mean_m) ** 2 for x in months) / len(months)
            std_m = math.sqrt(var_m)
            cv = (std_m / mean_m) if mean_m > 0 else 0.0
        else:
            cv = 0.0
        monthly_stability = 1.0 / (1.0 + cv)

        # recency: active days in last 30 days
        last_date = last.date()
        last30_cut = last_date.toordinal() - 30
        # Convert string keys to integers for comparison
        active_days_last_30d = sum(1 for d_ord_str in c['daily_flags'].keys() if int(d_ord_str) >= last30_cut)

        # Get POI info from cached data
        poi_info = c.get('poi_info', None)
        poi_available = c.get('poi_calculated', False) and poi_info is not None

        # Simple duration info from the simplified calculation
        duration_info = {
            'est_duration': c.get('est_duration', 0),
            'hourly_minutes': c.get('hourly_minutes', {})
        }
            
        return {
            'meta': {
                'first_seen': first.isoformat() if first else None,
                'last_seen': last.isoformat() if last else None,
                'span_days': span_days,
                'mean_coordinate': [c.get('mean_lat', None), c.get('mean_lon', None)] if c.get('mean_lat') is not None else None,
            },
            'level_1_primary': {
                'pings': visits,
                'unique_days': unique_days,
                'active_day_ratio': active_day_ratio,
                'gap_bins': dict(c['gap_bins']),
            },
            'level_2_secondary': {
                'hourly_hist': list(c['hourly_hist']),
                'weekday_hist': list(c['weekday_hist']),
                'monthly_hist': dict(c['monthly_hist']),
                'night_ratio': night_ratio,
                'weekday_day_ratio': weekday_day_ratio,
                'weekend_ratio': weekend_ratio,
                'midday_weekday_ratio': midday_weekday_ratio,
                'evening_ratio': evening_ratio,
                'early_late_overlap_day_ratio': late_evening_days_ratio,  # kept for backward compat
                'night_days_ratio': night_days_ratio,
                'weekday_work_days_ratio': weekday_work_days_ratio,
                'late_evening_days_ratio': late_evening_days_ratio,
                'early_morning_days_ratio': early_morning_days_ratio,
                'entropy_hour_norm': entropy_hour_norm,
                'monthly_stability': monthly_stability,
                'active_days_last_30d': active_days_last_30d,
            },
            'level_3_tertiary': {
                "poi_available": poi_available,
                "poi_info": poi_info
            },
            'level_4_duration': duration_info,
            'level_5_flux': {
                'flux_counts': dict(c.get('flux_counts', {'B': 0, 'C': 0, 'D': 0, 'E': 0, 'F': 0}))
            }
        }
    
    @staticmethod
    def score_home(ev: dict, a: float = 2.0) -> float:
        l1, l2 = ev['level_1_primary'], ev['level_2_secondary']
        visits = l1['pings']
        days = l1['unique_days']

        # Bayesian shrinkage priors
        p0_night = 8.0 / 24.0            # baseline night share
        night_ratio_shrunk = EvidenceStore._shrink_ratio(l2['night_ratio'], visits, p0_night, a=a)

        # sample-size weights
        w_visits = 1.0 - math.exp(-visits / 5.0)
        w_days = 1.0 - math.exp(-days / 3.0)    

        base = (
            0.375 * l2['night_days_ratio']
          + 0.10 * night_ratio_shrunk
          + 0.15 * l2['late_evening_days_ratio']
          + 0.10 * l2['early_morning_days_ratio']  # Added early morning evidence
          + 0.075 * (1.0 - l2['entropy_hour_norm'])
          + 0.25 * l1['active_day_ratio']  # Reduced from 0.30 to accommodate new features
          + 0.05 * l2['monthly_stability']  # Added monthly stability
        )
        s = base * w_visits * w_days

        # recency boost
        s *= min(1.0, 0.5 + 0.5 * (l2['active_days_last_30d'] / 10))
        return max(0.0, min(1.0, s))

    @staticmethod
    def score_work(ev: dict, a: float = 2.0) -> float:
        l1, l2 = ev['level_1_primary'], ev['level_2_secondary']
        visits = l1['pings']
        days = l1['unique_days']

        # Bayesian shrinkage priors
        p0_wd_day = 45.0 / 168.0         # (5 weekdays * 9 hours) / (7 days * 24 hours)
        weekday_day_ratio_shrunk = EvidenceStore._shrink_ratio(l2['weekday_day_ratio'], visits, p0_wd_day, a=a)

        # sample-size weights
        w_visits = 1.0 - math.exp(-visits / 5.0)
        w_days = 1.0 - math.exp(-days / 3.0)    

        base = (
            0.425 * l2['weekday_work_days_ratio']
          + 0.15 * weekday_day_ratio_shrunk
          + 0.10 * l2['midday_weekday_ratio']
          + 0.075 * (1.0 - l2['entropy_hour_norm'])
          + 0.20 * l1['active_day_ratio']  # Reduced from 0.25 to accommodate new features
          + 0.05 * l2['monthly_stability']  # Added monthly stability
        )
        s = base * w_visits * w_days

        # recency boost
        s *= min(1.0, 0.5 + 0.5 * (l2['active_days_last_30d'] / 10))
        return max(0.0, min(1.0, s))

    @staticmethod
    def score_leisure(ev: dict, a: float = 2.0) -> float:
        l1, l2 = ev['level_1_primary'], ev['level_2_secondary']
        visits = l1['pings']
        days = l1['unique_days']

        # Calculate home and work scores to invert them for leisure
        home_score = EvidenceStore.score_home(ev, a)
        work_score = EvidenceStore.score_work(ev, a)
        
        # Combine and invert home and work patterns
        combined_home_work = (home_score + work_score) / 2
        inverse_pattern = 1.0 - combined_home_work
        
        # Bayesian shrinkage priors
        p0_weekend = 2.0 / 7.0           # baseline weekend share
        weekend_ratio_shrunk = EvidenceStore._shrink_ratio(l2['weekend_ratio'], visits, p0_weekend, a=a)
        
        # Add evening_ratio (18-21h) with shrinkage
        p0_evening = 4.0 / 24.0
        evening_ratio_shrunk = EvidenceStore._shrink_ratio(l2['evening_ratio'], visits, p0_evening, a=a)
    
    

        # sample-size weights
        w_visits = 1.0 - math.exp(-visits / 5.0)
        w_days = 1.0 - math.exp(-days / 3.0)    
        
        base = (
            0.25 * weekend_ratio_shrunk
          + 0.20 * evening_ratio_shrunk
          + 0.15 * (1-l2['entropy_hour_norm'])
          + 0.10 * (1.0 - l2['monthly_stability'])
          + 0.30 * inverse_pattern  # Add inverse pattern from home and work
        )
        s = base * w_visits * w_days

        # Relaxed recency boost (leisure may not be recent frequently)
        s *= min(1.0, 0.5 + 0.5 * (l2['active_days_last_30d'] / 15))
        return max(0.0, min(1.0, s))
    
    def overall_score(self, ev: dict, a: float = 2.0) -> dict:
        home_score = self.score_home(ev, a)
        work_score = self.score_work(ev, a)
        leisure_score = self.score_leisure(ev, a)
        overall_score = {
            'home': home_score,
            'work': work_score,
            'leisure': leisure_score
        }
        if ev['level_3_tertiary']['poi_available']:
            poi_info = ev['level_3_tertiary']['poi_info']
            if poi_info:
                poi_label = poi_info['primary_category']
                if poi_label != 'path':
                    if poi_label in overall_score:
                        overall_score[poi_label] = 0.25 * overall_score[poi_label] + 0.75 * poi_info.get('confidence', 0.01) / 100.0
                elif poi_label == 'path':
                    overall_score[poi_label] = poi_info.get('confidence', 0.01) / 100.0

        return overall_score

In [28]:
store= EvidenceStore()

In [29]:
import pygeohash as pgh
df['geohash']=df.apply(lambda x: pgh.encode(x.latitude,x.longitude,precision=7),axis=1)

In [30]:
store = EvidenceStore()
rows = df[['geohash', 'timestamp', 'latitude', 'longitude', 'flux']].values.tolist()

# Process data
setin, coordinates, flux_values = {}, {}, {}
for gh, ts, lat, lon, flux in rows:
    if gh not in coordinates:
        coordinates[gh] = (lat, lon)
        flux_values[gh] = [flux] if flux else []
    else:
        coordinates[gh] = ((coordinates[gh][0] + lat)/2, (coordinates[gh][1] + lon)/2)
        if flux: flux_values.setdefault(gh, []).append(flux)
        
for gh, ts, _, _, _ in rows:
    setin.setdefault(gh, []).append(ts)

store.update(setin, coordinates, flux_values)

In [31]:
for i in store.store.keys():
    dr=store.derive(i)
    print(i,store.overall_score(dr))

ev3jj9z {'home': 0.5111081876969926, 'work': 0.29746026474663345, 'leisure': 0.3440354666409779}
ev3jj9w {'home': 0.2793850309168118, 'work': 0.1398404973167126, 'leisure': 0.35375589256128875}
ev0fhkq {'home': 0.42616127686136734, 'work': 0.17216095269551912, 'leisure': 0.3337362777830844}
ev0fhkn {'home': 0.07900946755114556, 'work': 0.07777005642113032, 'leisure': 0.16647848630536863}
evd7cyu {'home': 0.17319930063109085, 'work': 0.23154986348796208, 'leisure': 0.22886475154069588}
ev3jj9c {'home': 0.14073655366698531, 'work': 0.08159039474585614, 'leisure': 0.1451268669894757}
ey51kqf {'home': 0.07805072217779006, 'work': 0.28868974898652094, 'leisure': 0.2067594655703368}
ey51ucs {'home': 0.3010316832790599, 'work': 0.23333733851798072, 'leisure': 0.3260214039909962}
evd7cwp {'home': 0.4471519335866862, 'work': 0.34172496698298266, 'leisure': 0.2565267601769727}
ey7gkp8 {'home': 0.03605735696006747, 'work': 0.1162194246167697, 'leisure': 0.09176189283727641}
ey7gs0z {'home': 0.275

In [26]:
store.store['ev3jj9z']

{'pings': 3835,
 'first_seen_ts': Timestamp('2025-07-29 01:49:28+0000', tz='UTC'),
 'last_seen_ts': Timestamp('2025-09-17 19:03:47+0000', tz='UTC'),
 'unique_days': {datetime.date(2025, 7, 29),
  datetime.date(2025, 7, 31),
  datetime.date(2025, 8, 1),
  datetime.date(2025, 8, 2),
  datetime.date(2025, 8, 3),
  datetime.date(2025, 8, 5),
  datetime.date(2025, 8, 28),
  datetime.date(2025, 8, 29),
  datetime.date(2025, 8, 30),
  datetime.date(2025, 8, 31),
  datetime.date(2025, 9, 2),
  datetime.date(2025, 9, 3),
  datetime.date(2025, 9, 4),
  datetime.date(2025, 9, 5),
  datetime.date(2025, 9, 7),
  datetime.date(2025, 9, 8),
  datetime.date(2025, 9, 9),
  datetime.date(2025, 9, 10),
  datetime.date(2025, 9, 11),
  datetime.date(2025, 9, 12),
  datetime.date(2025, 9, 13),
  datetime.date(2025, 9, 14),
  datetime.date(2025, 9, 15),
  datetime.date(2025, 9, 16),
  datetime.date(2025, 9, 17)},
 'hourly_hist': [479,
  370,
  379,
  410,
  178,
  171,
  355,
  618,
  102,
  128,
  2,
  0,
 

In [13]:
# Decode the geohash 'ev3jj9z' to get latitude and longitude
lat, lon = pgh.decode('ev3jj9z')

# Plot the point on OpenStreetMap using plotly
import plotly.express as px

# Create a DataFrame for plotting
import pandas as pd
df_point = pd.DataFrame({'lat': [lat], 'lon': [lon]})

# Plot the point using plotly's scatter_mapbox
fig = px.scatter_mapbox(
    df_point,
    lat='lat',
    lon='lon',
    zoom=12,
    height=400,
    width=600
)
# Set the map style to OpenStreetMap
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

# Show the plot
fig.show()


/tmp/ipykernel_4782/322246661.py:12: DeprecationWarning: *scatter_mapbox* is deprecated! Use *scatter_map* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/
  fig = px.scatter_mapbox(


In [3]:
import duckdb
duckdb.query("SELECT * FROM 'data/raw_rt/**/data_e_*.parquet' limit 100").df()

,maid,timestamp,date,country,latitude,longitude,horizontal_accuracy,ipv4,ipv6,altitude,altitude_accuracy,flux
0,HhCyQ4RFvHcpySyMQcQYyYwObDQj2zwlDx0M/oxJa2vnU0...,2025-09-05 04:09:32+07:00,2025-09-04,ARE,24.1900,55.7623,0.0,92.99.52.32,,0.0,0.0,E
1,HhCyQ4RFvHcpySyMQcQYyYwObDQj2zwlDx0M/oxJa2vnU0...,2025-09-05 04:09:42+07:00,2025-09-04,ARE,24.1900,55.7623,0.0,92.99.52.32,,0.0,0.0,E
2,HhCyQ4RFvHcpySyMQcQYyYwObDQj2zwlDx0M/oxJa2vnU0...,2025-09-05 04:10:42+07:00,2025-09-04,ARE,24.1900,55.7623,0.0,92.99.52.32,,0.0,0.0,E
3,HhCyQ4RFvHcpySyMQcQYyYwObDQj2zwlDx0M/oxJa2vnU0...,2025-09-05 04:11:02+07:00,2025-09-04,ARE,24.1900,55.7623,0.0,92.99.52.32,,0.0,0.0,E
4,HhCyQ4RFvHcpySyMQcQYyYwObDQj2zwlDx0M/oxJa2vnU0...,2025-09-05 04:11:22+07:00,2025-09-04,ARE,24.1900,55.7623,0.0,92.99.52.32,,0.0,0.0,E
...,...,...,...,...,...,...,...,...,...,...,...,...
95,29BgfEOjx+7OFoNKQ+4Ei3S5/V1J9guyu2xf+L6SlcgXv1...,2025-09-05 04:47:11+07:00,2025-09-04,ARE,25.0734,55.2979,0.0,94.204.226.255,,0.0,0.0,E
96,29BgfEOjx+7OFoNKQ+4Ei3S5/V1J9guyu2xf+L6SlcgXv1...,2025-09-05 04:47:11+07:00,2025-09-04,ARE,25.1300,55.2100,0.0,94.204.226.255,,0.0,0.0,E
97,29BgfEOjx+7OFoNKQ+4Ei3S5/V1J9guyu2xf+L6SlcgXv1...,2025-09-05 04:47:11+07:00,2025-09-04,ARE,25.1300,55.2100,0.0,94.204.226.255,,0.0,0.0,E
98,29BgfEOjx+7OFoNKQ+4Ei3S5/V1J9guyu2xf+L6SlcgXv1...,2025-09-05 04:47:11+07:00,2025-09-04,ARE,25.1300,55.2100,0.0,94.204.226.255,,0.0,0.0,E


In [4]:
import pandas as pd
df=pd.read_parquet('/home/hieu/Work/new_casacom/data/months/all_maid.parquet')

In [17]:
df.maid.nunique()

8922016

In [18]:
df[df.flux.isin(['C','D','E'])].maid.nunique()

8795200

In [7]:
df1=pd.read_feather('/home/hieu/Work/maid_mapping.feather')

In [10]:
df1.month.unique()

array(['2025-08', '2025-05', '2025-04', '2025-07', '2025-06', '2025-03'],
      dtype=object)

In [12]:
df1.maid.nunique()

8954449

In [14]:
df1.head()

,maid,month,safe
0,TTdxrgaVIbY0W9ULiquW/9iRS+NA+6TdWJd5BMutLPTkNE...,2025-08,TTdxrgaVIbY0W9ULiquW_9iRS_NA_6TdWJd5BMutLPTkNE...
1,X8SLpLSFLkOdDjOCj4CpjUpYFMmCmGh8gjywGnWb5nPet8...,2025-08,X8SLpLSFLkOdDjOCj4CpjUpYFMmCmGh8gjywGnWb5nPet8...
2,/cQzYjWmshil6Rk38LPmpZTvoh5OGkDZzURiTna6gE7HCO...,2025-08,_cQzYjWmshil6Rk38LPmpZTvoh5OGkDZzURiTna6gE7HCO...
3,4OlT7YRwhHmjeFJWCzIzQLgHeadeoVMP+9Z93pe4tXyu53...,2025-08,4OlT7YRwhHmjeFJWCzIzQLgHeadeoVMP_9Z93pe4tXyu53...
4,8OZPOS6Rw5gQgpJOsWANid73RR3udNI4sIMdDXceX9H1RO...,2025-08,8OZPOS6Rw5gQgpJOsWANid73RR3udNI4sIMdDXceX9H1RO...


In [16]:
list_date=['2025-08']
df1[df1.month.isin(list_date)].maid.nunique()

5695450

In [19]:
pd.read_parquet('/home/hieu/Work/new_casacom/data/processed/2025-03-23.parquet')

,maid,flux,maid_flux
0,zaIJagdgzqGINiQKVBvFaqtD3fEz8r23oq+jc4cy7cMiC9...,D,1
1,kMX4HUtAen108ecyy68aXYpkItc/37McA2mQnxtn5MHNLu...,D,332
2,KHRD0L8lezDmPOk2wjMHG6er81WNhSoLvlCsEnSz5SC5Qh...,D,267
3,0zf6bBZm4MkaWY7VQEE9hIbCxR4Kilh/EQFrEPE1IxZpdj...,D,1
4,60Q06hblYdpSFwIH58ICEiaEKa+NBR/a8792buhTSnaNFJ...,D,467
...,...,...,...
1347844,9bkXBEUyLSUGVacWZBRHArg2eD39eosfeUXakL9CFVEaUG...,E,1
1347845,uBNRXVKKhfMqCeuRKhfQRmqWbAi7rzqFliPiRJ8rJ6gb1A...,E,2
1347846,Is4f2v4zzZiKtfd4JmnUuxhXy5aiIDc4ckW9TzEiNDRdSk...,E,2
1347847,kdBExJIS0A1juFMhUGJtSjF2278zj9aUJmwXktJNrOfCX6...,E,1


In [2]:
import pandas as pd
maid_tuple=pd.read_feather('/home/hieu/Work/new_casacom/maid_tuple.feather')
maid_tuple.head()


,0,1
0,000EBD/XIXbBin6HQ08gmg+OQwrvHB3x3GG3YGetOfNytm...,RLKOBff0zvdjFF8LMbCBLPOVgO35h8oD7XILk5Mh151w69...
1,000MWM0/Hb6Tx8kC8pN1NWkB0A8U5H3ISk73KqNWBjzI/r...,TrY9XNDbTt5O1iy+rzpX4hXfqTJM9qVPjX6KUWODixFJti...
2,000QpTCewIMVbIqUiH579haLp4qWc4Iy4Z4BjbJug8f32t...,q5LjUhhIsMUbiQ7DMRMczvQnFvUYGVeXC4aq+kMyl21Hrn...
3,000Vxw/LIJYZutqY8neYbgi5OXV7+Ly8HBJS8S1EeV77R/...,xqpE0/r/OZJTDVWrNgm+tn5FEDYuioe+wkq6fwGAfbt86L...
4,000z0CwQ/RA+Q7DIGRkyQJBfuuufOomA/t6lfV7G96+PuC...,W+i/V5/d7p8wPR5pHidHKoy8VSaLtagUUL5pJ1yoeSDGZk...


In [3]:
import duckdb
a=duckdb.query(
"""
    select * from read_csv('/home/hieu/Work/new_casacom/result/home.csv',max_line_size=12037090)
    where maid in (select "0" from maid_tuple)
"""
).df()
b=duckdb.query(
"""
    select * from read_csv('/home/hieu/Work/new_casacom/result/home.csv',max_line_size=12037090)
    where maid in (select "1" from maid_tuple)
"""
).df()


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [60]:
# pd.set_option('display.max_columns', None)
# rs=duckdb.query(
# f"""
#     select * from read_csv(['/home/hieu/Work/new_casacom/result/home.csv','/home/hieu/Work/new_casacom/result/work.csv','/home/hieu/Work/new_casacom/result/leisure.csv'],max_line_size=12037090)
#     where maid in {d}
# """
# ).df()

In [86]:
id=12
res=maid_tuple[maid_tuple[0]==a.maid.unique()[id]]
d=[]
d+=res[0].to_list()
d+=res[1].to_list()

In [87]:
pd.set_option('display.max_columns', None)
rs=duckdb.query(
f"""
    select * from read_csv('/home/hieu/Work/new_casacom/result/home.csv',max_line_size=12037090)
    where maid in {d}
"""
).df()

In [88]:
rs

,maid,geohash,category,confidence,country,lat,lon,first_seen,last_seen,est_duration,pings,unique_days,evidence_score,fluxB,fluxC,fluxD,fluxE,fluxF,monday,tuesday,wednesday,thursday,friday,saturday,sunday,month_1,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12,hour_0,hour_1,hour_2,hour_3,hour_4,hour_5,hour_6,hour_7,hour_8,hour_9,hour_10,hour_11,hour_12,hour_13,hour_14,hour_15,hour_16,hour_17,hour_18,hour_19,hour_20,hour_21,hour_22,hour_23,poi_score,poi_info,home,work,leisure,path,maid_total_pings,cell_info,maid_fluxB,maid_fluxC,maid_fluxD,maid_fluxE,maid_fluxF,maid_countries,maid_countries_pings,maid_first_seen,maid_last_seen
0,siU6cO0pvRX7sEmbgu3jicTNxTmR96pqGWCnPY7Xfoh7AZ...,th35mx9,home,0.397094,Saudi Arabia,24.520000,46.650000,2025-04-01 08:48:12,2025-04-26 11:59:39,202,39,18,"{'home': 0.39709408517366374, 'work': 0.367883...",0,0,0,39,0,1,3,4,3,2,3,2,0,0,0,18,0,0,0,0,0,0,0,0,0,3,4,3,1,0,1,1,1,5,1,3,0,2,8,0,0,0,4,1,1,0,0,0,None,None,0.397094,0.367884,0.331961,0.0,257,"{'th35mx9': {'distance': 0, 'radio': 'unknown'...",0,0,0,100,0,{'Saudi Arabia'},{'Saudi Arabia': 100},2025-04-01 08:48:12,2025-08-29 01:33:01
1,siU6cO0pvRX7sEmbgu3jicTNxTmR96pqGWCnPY7Xfoh7AZ...,th35vn6,home,0.377261,Saudi Arabia,24.600000,46.630000,2025-05-06 22:08:50,2025-08-29 01:33:01,288,61,28,"{'home': 0.37726081032444747, 'work': 0.158360...",0,0,0,61,0,3,3,4,4,6,4,4,0,0,0,0,10,6,6,6,0,0,0,0,0,8,6,2,11,9,4,0,0,0,4,0,0,2,1,5,1,2,3,0,1,2,0,0,None,None,0.377261,0.158360,0.257807,0.0,257,"{'th35vn6': {'distance': 0, 'radio': 'unknown'...",0,0,0,100,0,{'Saudi Arabia'},{'Saudi Arabia': 100},2025-04-01 08:48:12,2025-08-29 01:33:01
2,bC6zxuaBJgOZ/KINsdgOrF5R6UKckx2TcfrVDmSk6pp4s6...,th35epx,home,0.796018,Saudi Arabia,24.563242,46.548291,2025-08-19 16:50:17,2025-09-15 23:06:01,647,3606,25,"{'home': 0.7960181454283225, 'work': 0.5071161...",1568,1480,278,0,280,2,4,4,4,4,4,3,0,0,0,0,0,0,0,11,14,0,0,0,308,186,149,49,80,196,77,245,49,169,123,266,392,505,159,63,4,16,4,68,132,133,107,126,None,None,0.796018,0.507116,0.252620,0.0,4711,"{'th35epx': {'distance': 0, 'radio': 'unknown'...",1568,1480,278,1,280,"{'Morocco', 'Saudi Arabia'}","{'Saudi Arabia': 3606, 'Morocco': 1}",2025-06-26 03:08:29,2025-09-15 23:06:01


In [91]:
pd.set_option('display.max_columns', None)
duckdb.query(
f"""
    select * from read_csv(['/home/hieu/Work/new_casacom/results/home_final.csv','/home/hieu/Work/new_casacom/results/work_final.csv','/home/hieu/Work/new_casacom/results/leisure_final.csv'],max_line_size=12037090)
    where maid in {d}
"""
).df()

,maid,geohash,category,confidence,country,lat,lon,first_seen,last_seen,est_duration,pings,unique_days,evidence_score,fluxB,fluxC,fluxD,fluxE,fluxF,monday,tuesday,wednesday,thursday,friday,saturday,sunday,month_1,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12,hour_0,hour_1,hour_2,hour_3,hour_4,hour_5,hour_6,hour_7,hour_8,hour_9,hour_10,hour_11,hour_12,hour_13,hour_14,hour_15,hour_16,hour_17,hour_18,hour_19,hour_20,hour_21,hour_22,hour_23,poi_score,poi_info,home,work,leisure,path,maid_total_pings,cell_info,maid_fluxB,maid_fluxC,maid_fluxD,maid_fluxE,maid_fluxF,maid_countries,maid_countries_pings,maid_first_seen,maid_last_seen
0,siU6cO0pvRX7sEmbgu3jicTNxTmR96pqGWCnPY7Xfoh7AZ...,th35epx,home,0.794508,Saudi Arabia,24.563251,46.548265,2025-08-19 16:50:17,2025-09-15 23:06:52,663,2042,25,"{'home': 0.794508140405783, 'work': 0.50323110...",942,736,139,0,225,3,3,4,3,4,4,4,0,0,0,0,0,0,0,10,15,0,0,0,29,43,108,41,131,32,93,85,154,213,293,108,39,2,10,2,41,73,71,57,70,166,95,86,{},None,0.794508,0.503231,0.247396,0.0,2785,"{'th35epx': {'distance': 0, 'radio': 'unknown'...",942,736,139,2,225,"{'Morocco', 'Saudi Arabia'}","{'Saudi Arabia': 2042, 'Morocco': 2}",2025-06-26 04:16:50,2025-09-15 23:06:52


In [93]:
pd.set_option('display.max_columns', None)
duckdb.query(
f"""
    select * from read_csv('/home/hieu/Work/new_casacom/results/path_new_final.csv',max_line_size=12037090)
    where maid in {d}
"""
).df()

,maid,geohash,category,confidence,country,lat,lon,first_seen,last_seen,est_duration,pings,unique_days,evidence_score,fluxB,fluxC,fluxD,fluxE,fluxF,monday,tuesday,wednesday,thursday,friday,saturday,sunday,month_1,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12,hour_0,hour_1,hour_2,hour_3,hour_4,hour_5,hour_6,hour_7,hour_8,hour_9,hour_10,hour_11,hour_12,hour_13,hour_14,hour_15,hour_16,hour_17,hour_18,hour_19,hour_20,hour_21,hour_22,hour_23,poi_score,poi_info,home,work,leisure,path,maid_total_pings,cell_info,maid_fluxB,maid_fluxC,maid_fluxD,maid_fluxE,maid_fluxF,maid_countries,maid_countries_pings,maid_first_seen,maid_last_seen
0,siU6cO0pvRX7sEmbgu3jicTNxTmR96pqGWCnPY7Xfoh7AZ...,th35epx,home,0.794508,Saudi Arabia,24.563251,46.548265,2025-08-19 16:50:17,2025-09-15 23:06:52,663,2042,25,"{'home': 0.794508140405783, 'work': 0.50323110...",942,736,139,0,225,3,3,4,3,4,4,4,0,0,0,0,0,0,0,10,15,0,0,0,29,43,108,41,131,32,93,85,154,213,293,108,39,2,10,2,41,73,71,57,70,166,95,86,{},None,0.794508,0.503231,0.247396,0.00000,2785,"{'th35epx': {'distance': 0, 'radio': 'unknown'...",942,736,139,2,225,"{'Morocco', 'Saudi Arabia'}","{'Saudi Arabia': 2042, 'Morocco': 2}",2025-06-26 04:16:50,2025-09-15 23:06:52
1,siU6cO0pvRX7sEmbgu3jicTNxTmR96pqGWCnPY7Xfoh7AZ...,ey51k5e,path,0.966750,Morocco,33.990000,-6.850000,2025-06-26 04:16:50,2025-07-02 23:21:10,0,2,2,"{'home': 0.06580054123743027, 'work': 0.013032...",0,0,0,2,0,0,0,1,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,{'path': 0.966749574593662},"{'osm_id': 456777787, 'foot': 'yes', 'highway'...",0.065801,0.013032,0.041619,0.96675,2785,"{'ey51k50': {'distance': 487.393857060244, 'ra...",942,736,139,2,225,"{'Morocco', 'Saudi Arabia'}","{'Saudi Arabia': 2042, 'Morocco': 2}",2025-06-26 04:16:50,2025-09-15 23:06:52


In [ ]:

# Test Canonical MAID Mapping System

import pandas as pd
import os

# Load MAID tuple mapping từ file feather (giống như trong process_maroc.py)
maid_tuple_file = os.environ.get('MAID_TUPLE_FILE', './maid_tuple.feather')
print(f"Loading MAID tuple mapping from {maid_tuple_file}...")

# Read the maid_tuple file (first few rows for testing)
maid_tuple = pd.read_feather(maid_tuple_file)
print(f"Loaded {len(maid_tuple)} MAID pairs for testing")

# Show first 10 rows to understand the structure
print("\nFirst 10 MAID pairs:")
for i, (_, row) in enumerate(maid_tuple.head(10).iterrows()):
    canonical_maid = row[0]
    duplicate_maid = row[1]
    print(f"  {i+1}. Canonical: {canonical_maid[:50]}...")
    print(f"     Duplicate: {duplicate_maid[:50]}...")

# Create maid_mapping dictionary (giống như trong process_maroc.py)
maid_mapping = {}
valid_maids = set()

for _, row in maid_tuple.iterrows():
    canonical_maid = row[0]
    duplicate_maid = row[1]
    maid_mapping[canonical_maid] = canonical_maid
    maid_mapping[duplicate_maid] = canonical_maid
    valid_maids.add(canonical_maid)
    valid_maids.add(duplicate_maid)

print(f"\nCreated mapping for {len(maid_mapping)} MAIDs")
print(f"Total unique valid MAIDs: {len(valid_maids)}")

In [11]:
maid_list=list(maid_tuple[0].unique())+list(maid_tuple[1].unique())

In [12]:
df=pd.DataFrame(maid_list,columns=['maid'])

In [ ]:
import duckdb
conn=duckdb.connect()
conn.execute("SET timezone='UTC'")
maid_df=conn.query(
"""
    select DISTINCT ON (latitude, longitude) maid, timestamp, country, latitude, longitude, flux  
    from read_parquet('/home/hieu/Work/new_casacom/data/raw_rt/**/*.parquet')
"""
).df()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [22]:
data_maid_unique=maid_df[maid_df.maid.isin(df.maid.unique())].maid.unique()

In [67]:
# Use actual MAIDs from data instead of test_maids
test_maids = data_maid_unique
print(f"Testing with {len(test_maids)} MAIDs:")

# Create DataFrame from test_maids
sample_data = pd.DataFrame({
    'maid': test_maids
})

print("\nOriginal MAIDs:")
print(sample_data[['maid']].head(10))

# Apply canonical mapping
print("\nAfter canonical mapping:")
sample_data['canonical_maid'] = sample_data['maid'].map(maid_mapping)
print(sample_data[['maid', 'canonical_maid']].head(10))

# Count unique MAIDs before and after mapping
original_unique = sample_data['maid'].nunique()
canonical_unique = sample_data['canonical_maid'].dropna().nunique()

print(f"\nOriginal unique MAIDs: {original_unique}")
print(f"Canonical unique MAIDs: {canonical_unique}")
print(f"Reduction: {original_unique - canonical_unique} MAIDs consolidated")

# Show detailed mapping results
print("\nDetailed mapping results:")
for i, test_maid in enumerate(test_maids[:10]):
    if test_maid in maid_mapping:
        canonical_result = maid_mapping[test_maid]
        print(f"  {i+1}. {test_maid} → {canonical_result}")
    else:
        print(f"  {i+1}. {test_maid[:50]}... → NOT FOUND")

print("\n" + "="*60)
print("Canonical MAID system helps consolidate duplicate device IDs")
print("into unique canonical identifiers for consistent analysis.")
print("="*60)


Testing with 26800 MAIDs:

Original MAIDs:
                                                maid
0  ES5qgyL9mzqnWl8Kv/Tm1T8wmviKl12thvR7XPnRfcgqO3...
1  cIDkOoh/Qv3+jnUij0WPKn367hvxAPZtc4/pIOOgl8xaCN...
2  MzzaJTgK+kOvYXzJXvfNRKvG7eUezESy9yn/DGIWZUtOwK...
3  j2VAlhr1HxX/0UGHPMPz9uggcNo34wvoUbi0NRHFUeNnoA...
4  I5fDumPpzOTo/EqG92UnxgP1GT0yqMCSLoULHYlUsp9bSo...
5  6uXzaCpeLCEZLATT8rHB2WOEWxk10szkFccsuDi9mR9ufu...
6  /baaZ0+UFVPB4c+aIXf0tbLELEb80ONCvrzrH7vslAPTG/...
7  wkJ0p7r8k19E7UFuDu34uBr3HPA0OcAznNAtA50QmR7eUN...
8  rCCUnZcIbuwc7rIvDx8LQE0sVVqPCLLYurO65I09S7mMH7...
9  rbKSve0cxYwUBSElILzt4qflwqu61mFMjM/oeiIKi/+O2g...

After canonical mapping:
                                                maid  \
0  ES5qgyL9mzqnWl8Kv/Tm1T8wmviKl12thvR7XPnRfcgqO3...   
1  cIDkOoh/Qv3+jnUij0WPKn367hvxAPZtc4/pIOOgl8xaCN...   
2  MzzaJTgK+kOvYXzJXvfNRKvG7eUezESy9yn/DGIWZUtOwK...   
3  j2VAlhr1HxX/0UGHPMPz9uggcNo34wvoUbi0NRHFUeNnoA...   
4  I5fDumPpzOTo/EqG92UnxgP1GT0yqMCSLoULHYlUsp9bSo...   
5  6uXzaCpeL

In [33]:
dup_data=maid_df[maid_df.maid.isin(valid_maids)]

In [ ]:
dup_data.reset_index(drop=True,inplace=True)
dup_data['maid']=dup_data['maid'].map(maid_mapping)

In [59]:
origin=maid_df[maid_df.maid.isin(df.maid.unique())]

In [61]:
import pygeohash as pgh
cleaned_data=origin[((origin['latitude'] >= -90.0) & 
                        (origin['latitude'] <= 90.0) & 
                        (origin['longitude'] >= -180.0) & 
                        (origin['longitude'] <= 180.0) &
                        (origin['latitude'].notna()) &
                        (origin['longitude'].notna()) &
                        (~origin['latitude'].isin([float('inf'), float('-inf')]) &
                        (~origin['longitude'].isin([float('inf'), float('-inf')]))))]
cleaned_data['geohash']=cleaned_data.apply(lambda x: pgh.encode(x.latitude,x.longitude,precision=7),axis=1)

/tmp/ipykernel_4382/27797046.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_data['geohash']=cleaned_data.apply(lambda x: pgh.encode(x.latitude,x.longitude,precision=7),axis=1)


In [62]:
cleaned_data.head()

,maid,timestamp,country,latitude,longitude,flux,geohash
26,ES5qgyL9mzqnWl8Kv/Tm1T8wmviKl12thvR7XPnRfcgqO3...,2025-08-01 00:40:48+00:00,CAN,46.071621,-71.940353,B,f2hy4gn
29,cIDkOoh/Qv3+jnUij0WPKn367hvxAPZtc4/pIOOgl8xaCN...,2025-07-29 22:23:20+00:00,CAN,45.645340,-73.863136,B,f257xjq
31,MzzaJTgK+kOvYXzJXvfNRKvG7eUezESy9yn/DGIWZUtOwK...,2025-08-01 13:39:55+00:00,CAN,43.665958,-79.371819,B,dpz83v5
32,j2VAlhr1HxX/0UGHPMPz9uggcNo34wvoUbi0NRHFUeNnoA...,2025-07-29 16:32:22+00:00,CAN,45.445938,-73.783722,B,f25d910
50,I5fDumPpzOTo/EqG92UnxgP1GT0yqMCSLoULHYlUsp9bSo...,2025-07-31 03:24:11+00:00,CAN,43.639946,-79.419975,B,dpz82b2


In [66]:
from process_maroc import *

In [63]:
black_list=pd.read_parquet('/home/hieu/Work/new_casacom/blacklist_geohash.parquet')
cleaned_data=cleaned_data[~cleaned_data['geohash'].isin(black_list)]
# Group cleaned data by maid
grouped_data = cleaned_data.groupby('maid')

maid_data_list = [(maid, group) for maid, group in grouped_data]
try:
    max_workers_env = int(os.environ.get('MAX_WORKERS', '0'))
except Exception:
    max_workers_env = 0
cpu_count = mp.cpu_count()
if max_workers_env and max_workers_env > 0:
    num_processes = min(len(maid_data_list), max_workers_env)
else:
    # Default to a conservative fraction to avoid I/O contention
    num_processes = min(len(maid_data_list), max(1, min(cpu_count, 96)))
# Chunksize for executor.map to reduce overhead
try:
    map_chunksize = int(os.environ.get('MAP_CHUNKSIZE', '100'))
except Exception:
    map_chunksize = 100

    
with ProcessPoolExecutor(max_workers=num_processes) as executor:
    results = list(tqdm(
        executor.map(process_with_output_dir, maid_data_list, chunksize=map_chunksize), 
        total=len(maid_data_list), 
        desc=f"Processing MAIDs"
    ))

# Count successful results
successful_maids = sum(1 for _, success in results if success)

print(f"Completed processing {successful_maids} MAIDs")

# Close DuckDB connection for this date
conn.close()

# Collect garbage after processing each date to free up memory
gc.collect()


Processing MAIDs: 100%|██████████| 26800/26800 [01:10<00:00, 381.43it/s]


Completed processing 26800 MAIDs


605

In [64]:
import glob
import os
all_maid=glob.glob('*.pkl')

In [65]:
# Ensure the target directory exists
target_dir = 'data/test_maid/'
os.makedirs(target_dir, exist_ok=True)

# Move all pickle files to the target directory
for pkl_file in all_maid:
    source_path = pkl_file
    target_path = os.path.join(target_dir, os.path.basename(pkl_file))
    if os.path.exists(source_path):
        os.rename(source_path, target_path)

len(all_maid)

26800

In [109]:
a = [
    'ES5qgyL9mzqnWl8Kv/Tm1T8wmviKl12thvR7XPnRfcgqO3/Z76odN2wW++L0i8DE',
    'cIDkOoh/Qv3+jnUij0WPKn367hvxAPZtc4/pIOOgl8xaCN1AVmhyDS7j/rRHtzwM',
    'MzzaJTgK+kOvYXzJXvfNRKvG7eUezESy9yn/DGIWZUtOwKzYQ5rML4pADnYhc9uc',
    'j2VAlhr1HxX/0UGHPMPz9uggcNo34wvoUbi0NRHFUeNnoARBUnoL6A0N0Whi+5rt',
    'I5fDumPpzOTo/EqG92UnxgP1GT0yqMCSLoULHYlUsp9bSoqYGtjcjwfa46mv/gu7',
    '6uXzaCpeLCEZLATT8rHB2WOEWxk10szkFccsuDi9mR9ufuzhyb4IjGQWOWPybfVR',
    '/baaZ0+UFVPB4c+aIXf0tbLELEb80ONCvrzrH7vslAPTG/lYMIE1lF+0ToGfOiN7',
    'wkJ0p7r8k19E7UFuDu34uBr3HPA0OcAznNAtA50QmR7eUNC2qo83UGuH9SbNvncl',
    'rCCUnZcIbuwc7rIvDx8LQE0sVVqPCLLYurO65I09S7mMH7G42euEt/DiBtZfkzQk',
    'rbKSve0cxYwUBSElILzt4qflwqu61mFMjM/oeiIKi/+O2gy1M2OTWFUE5vJ4DDKk'
]

b = [
    'PKTKvoFO+hiA4qiWXVmOjchdUI+TsOgnF2yEraP6LVfJ/DluYdtYnaC/BIzuCLKm',
    'CsrviXAeDI9mlr/kXUB8OH5ImGi1357pv3x3mgFE/eDOt4OR1rp2Nyk3lWFQTHWQ',
    'ePYoHFzodpeoK8UCgbytNP6wwpZifWrZ/ck/FcHILDeNhUKvJMGnjr1F5IrZOZUp',
    '19hSkpienGhTtovF4vztHbeqU0Madejl0WSCdESDgJ7NAgQ0U04/9clOlD1vdTDK',
    'Vs3lBhH2PXDi6a6trFNnyVgk8vZEI4uHB7BKyK6Av331XkwYnJk/gWfxmil7DEVN',
    'rH1mWpPzZKipjpI9AYF97cRa2mIlr+3jRsVswS7E6Gs55WAQi+VLu7oJk0MpiK/s',
    'RcHU3M6nq2ALkpgPZvlQEPiXGYUy22SRe9XlpSYzLSCj98u9pGgnfWqFonxn75f9',
    'InjsxP0U33VR9MnYHVNGTYAjyricW6Jfd3N8VoOXRhuosL8xaunSRVDckz2xSXOI',
    'GG3UnXDhOqRQqmzbgWXVnqc5Z1PrPv20snysp5+3LGUi66gy4JQ1Pzh03q50dADO',
    'aE2XwLrzDA+XZ5y2eMHR5EY4NsHPHyvX4T/cG39869tzyR1U93X4nr5DIbsqeILm'
]

In [84]:
import glob
p=glob.glob('/home/hieu/Work/new_casacom/data/processed/*.pkl')
p2=glob.glob('/home/hieu/Work/new_casacom/data/processed_2/*.pkl')
p_list=set([i.split('/')[-1][:-4] for i in p])
p2_list=set([i.split('/')[-1][:-4] for i in p2])

In [108]:
len(p2_list),len(p_list)

(36275, 34002)

In [135]:
for i in zip(a,b):
    try:
        a_store=pd.read_pickle(f'/home/hieu/Work/new_casacom/data/processed_2/{make_safe_filename(i[0])}')['store']
        b_store=pd.read_pickle(f'/home/hieu/Work/new_casacom/data/processed_2/{make_safe_filename(i[1])}')['store']
        print(make_safe_filename(i[0]),make_safe_filename(i[1]))
        print("equal")
        after_merge=pd.read_pickle(f'/home/hieu/Work/new_casacom/data/processed/{make_safe_filename(i[1])}')['store']
        break
    except:
        continue
    

wkJ0p7r8k19E7UFuDu34uBr3HPA0OcAznNAtA50QmR7eUNC2qo83UGuH9SbNvncl.pkl InjsxP0U33VR9MnYHVNGTYAjyricW6Jfd3N8VoOXRhuosL8xaunSRVDckz2xSXOI.pkl
equal


In [141]:

for i in a_store.keys():
    try:
        ping_a=a_store[i]['pings']
        ping_b=b_store[i]['pings']
        if ping_a!=ping_b:
            print(i)
            print(ping_a,ping_b)
            print('not equal')

    except:
        continue

6gmqzg4
4 1
not equal
7nwqt74
5 1
not equal
7pj6z8w
5 1
not equal
9g6njcd
2 1
not equal
9g3qp5n
5 1
not equal
9g9f2mb
3 1
not equal
9exz5q2
2 1
not equal
d58r28f
7 2
not equal
wydhjgx
2 1
not equal
9epnyku
2 1
not equal
teud55u
2 1
not equal
9ezk2tb
2 1
not equal
9u8fbj5
6 2
not equal
wyd6570
2 1
not equal
wyg3tsh
2 1
not equal
63pd758
2 1
not equal
r1x3h99
2 1
not equal
6tgh4j4
2 1
not equal
9g3n597
4 1
not equal
9ez8ptd
3 1
not equal
thets3b
2 1
not equal
9thkywy
4 1
not equal
sv9t8dy
3 2
not equal
wy7hr64
1 2
not equal
wy6wfdw
2 1
not equal
6fdxx2e
2 1
not equal
9g3qxpk
3 1
not equal
9g985e2
2 1
not equal
sv9tb5b
2 3
not equal
sv9qp36
1 2
not equal
wy60qdh
2 1
not equal
tqw646c
1 2
not equal
k3vnr9q
1 2
not equal
k3vnm7j
1 2
not equal
k3vr382
2 3
not equal
kd74e2y
1 4
not equal
ke7g1fp
2 1
not equal
9exbb2k
2 1
not equal
9ez8ntv
2 1
not equal
sv9td4c
2 4
not equal
svc3pv9
1 2
not equal
sy9fbsm
1 2
not equal
sxhswwh
1 2
not equal
69y7ssy
1 2
not equal
svztbux
1 2
not equal
tw1hms1
1 

In [145]:
a_store['d58r28f']

{'pings': 7,
 'first_seen_ts': Timestamp('2025-08-05 00:31:00+0000', tz='UTC'),
 'last_seen_ts': Timestamp('2025-08-07 05:28:57+0000', tz='UTC'),
 'unique_days': {datetime.date(2025, 8, 5), datetime.date(2025, 8, 7)},
 'hourly_hist': [4,
  0,
  0,
  0,
  0,
  3,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'weekday_hist': [0, 4, 0, 3, 0, 0, 0],
 'hourly_hist_weekday': [4,
  0,
  0,
  0,
  0,
  3,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'hourly_hist_weekend': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'monthly_hist': {'2025-08': 7},
 'daily_flags': {739468: 8, 739470: 9},
 'max_seen_date': datetime.date(2025, 8, 7),
 'gap_bins': {'0d': 0, '1-3d': 1, '4-7d': 0, '8-30d': 0, '>30d': 0},
 'poi_info': None,
 'poi_calculated': False,
 'mean_lat': 20.967070817947388,
 'mean_lon': -89.6236982345581,
 'coord_count': 4,
 '

In [146]:
b_store['d58r28f']

{'pings': 2,
 'first_seen_ts': Timestamp('2025-08-07 05:28:09+0000', tz='UTC'),
 'last_seen_ts': Timestamp('2025-08-08 02:35:32+0000', tz='UTC'),
 'unique_days': {datetime.date(2025, 8, 7), datetime.date(2025, 8, 8)},
 'hourly_hist': [0,
  0,
  1,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'weekday_hist': [0, 0, 0, 1, 1, 0, 0],
 'hourly_hist_weekday': [0,
  0,
  1,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'hourly_hist_weekend': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'monthly_hist': {'2025-08': 2},
 'daily_flags': {739470: 9, 739471: 8},
 'max_seen_date': datetime.date(2025, 8, 8),
 'gap_bins': {'0d': 0, '1-3d': 1, '4-7d': 0, '8-30d': 0, '>30d': 0},
 'poi_info': None,
 'poi_calculated': False,
 'mean_lat': 20.967,
 'mean_lon': -89.6237,
 'coord_count': 2,
 'hourly_minutes': {5: 

In [147]:
after_merge['d58r28f']

{'pings': 9,
 'first_seen_ts': Timestamp('2025-08-05 00:31:00+0000', tz='UTC'),
 'last_seen_ts': Timestamp('2025-08-08 02:35:32+0000', tz='UTC'),
 'unique_days': {datetime.date(2025, 8, 5),
  datetime.date(2025, 8, 7),
  datetime.date(2025, 8, 8)},
 'hourly_hist': [4,
  0,
  1,
  0,
  0,
  4,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'weekday_hist': [0, 4, 0, 4, 1, 0, 0],
 'hourly_hist_weekday': [4,
  0,
  1,
  0,
  0,
  4,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'hourly_hist_weekend': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'monthly_hist': {'2025-08': 9},
 'daily_flags': {739468: 8, 739470: 9, 739471: 8},
 'max_seen_date': datetime.date(2025, 8, 8),
 'gap_bins': {'0d': 0, '1-3d': 2, '4-7d': 0, '8-30d': 0, '>30d': 0},
 'poi_info': None,
 'poi_calculated': False,
 'mean_lat': 20.96705722579956,
 'mean_lon':